# Grad-TTS: A Diffusion Probabilistic Model for Text-to-Speech

Official implementation of the Grad-TTS model based on Diffusion Probabilistic Models. For all details check out our paper accepted to ICML 2021 via [this](https://arxiv.org/abs/2105.06337) link.

You can listen to the samples on our demo page via [this](https://grad-tts.github.io/) link.

You can access Google Colab demo notebook via [this](https://colab.research.google.com/drive/1YNrXtkJQKcYDmIYJeyX8s5eXxB4zgpZI?usp=sharing) link.

**Authors**: Vadim Popov\*, Ivan Vovk\*, Vladimir Gogoryan, Tasnima Sadekova, Mikhail Kudinov.

<sup>\*Equal contribution.</sup>

**Note**: for fast synthesis prefer running inference on GPU device.

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import argparse
import json
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm
from scipy.io.wavfile import write
from denoiser import Denoiser
from tqdm.auto import tqdm


import torch

# For Grad-TTS
import params
from model import GradTTS
from text import text_to_sequence, cmudict, sequence_to_text
from text.symbols import symbols
from utils import intersperse

# For HiFi-GAN
import sys
sys.path.append('./hifi-gan/')
from env import AttrDict
from models import Generator as HiFiGAN

In [4]:
N_SPKS = 1  # 247 for Libri-TTS model and 1 for single speaker (LJSpeech)

In [6]:
location = "logs/grad_det_new_cleaner/grad_2000.pt"

In [7]:
model_dict = torch.load(location, map_location=lambda loc, storage: loc)

generator = GradTTS(len(symbols)+1, N_SPKS, params.spk_emb_dim,
                    params.n_enc_channels, params.filter_channels,
                    params.filter_channels_dp, params.n_heads, params.n_enc_layers,
                    params.enc_kernel, params.enc_dropout, params.window_size,
                    params.n_feats, params.dec_dim, params.beta_min, params.beta_max,
                    pe_scale=1000, dur_pred_type=model_dict['params'].dur_pred_type)  # pe_scale=1 for `grad-tts-old.pt`
generator.load_state_dict(model_dict['model'])
_ = generator.cuda().eval()
print(f'Number of parameters: {generator.nparams}')

cmu = cmudict.CMUDict('./resources/cmu_dictionary')

Number of parameters: 14840792


In [8]:
with open('./checkpts/hifigan-config.json') as f:
    h = AttrDict(json.load(f))
hifigan = HiFiGAN(h)
hifigan.load_state_dict(torch.load('/home/smehta/Projects/CLIP-DIFF/generator_v1', map_location=lambda loc, storage: loc)['generator'])
_ = hifigan.cuda().eval()
hifigan.remove_weight_norm()
denoiser = Denoiser(hifigan, mode='zeros')
%matplotlib inline

Removing weight norm...


In [9]:
from pathlib import Path
import soundfile as sf

def save_to_folder(filename: str, output: torch.Tensor, folder: str):
    folder = Path(folder)
    folder.mkdir(exist_ok=True, parents=True)
    sf.write(folder / f'{filename}.wav', output, 22050, 'PCM_24')

In [10]:
texts = [
    "The position of this palmprint on the carton was parallel with the long axis of the box, and at right angles with the short axis;",
    "we will not allow ourselves to run around in new circles of futile discussion and debate, always postponing the day of decision.",
    "It is now possible for Presidents to travel the length and breadth of a land far larger than the United States",
    "This is proved by contemporary accounts, especially one graphic and realistic article which appeared in the 'Times,'",
    "A formal and thorough description of the responsibilities of the advance agent is now in preparation by the Service.",
    "The rentals of rooms and fees went to the warden, whose income was two thousand three hundred seventy-two pounds.",
    "with respect to industry and business, but nearly all are agreed that private enterprise in times such as these.",
    "The only exit from the office in the direction Oswald was moving was through the door to the front stairway.",
    "it was the beginning of the great collections of Babylonian antiquities in the museums of the Western world.",
    "The bank enjoyed an excellent reputation, it had a good connection, and was supposed to be perfectly sound.",
    "But he did not like the risk of entering a room by the fireplace, and the chances of detection it offered.",
    "Later on he had devoted himself to the personal investigation of the prisons of the United States.",
    "and the windows of the opposite houses, which commanded a good view, as usual fetched high prices.",
    "The boy declared he saw no one, and accordingly passed through without paying the toll of a penny.",
    "There should be at least a full and fair trial given to these means of ending industrial warfare;",
    ]

In [11]:
texts = [
    "The overwhelming majority of people in this country know how to sift the wheat from the chaff in what they hear and what they read.",
    "If somebody did that to me, a lousy trick like that, to take my wife away, and all the furniture, I would be mad as hell, too.",
#     "as is shown by the report of the Commissioners to inquire into the state of the municipal corporations in 1835.",
    "Even the Caslon type when enlarged shows great shortcomings in this respect:",
    "All the committee could do in this respect was to throw the responsibility on others.",
    "These pungent and well grounded strictures applied with still greater force to the unconvicted prisoner, the man who came to the prison innocent, and still uncontaminated.",
    "and recognized as one of the frequenters of the bogus law stationers. His arrest led to that of others.",
    "Oswald was, however, willing to discuss his contacts with Soviet authorities. He denied having any involvement with Soviet intelligence agencies",
    "The first physician to see the President at Parkland Hospital was Dr. Charles J. Carrico, a resident in general surgery.",
    "during the morning of November 22 prior to the motorcade.",
    "On occasion the Secret Service has been permitted to have an agent riding in the passenger compartment with the President.",
    "although at Mr. Buxton's visit a new jail was in process of erection, the first step towards reform since Howard's visitation in 1774.",
    "or theirs might be one of many, and it might be considered necessary to make an example.",
    "The Warren Commission Report. By The President's Commission on the Assassination of President Kennedy. Chapter 7. Lee Harvey Oswald.",
    "Mr. Wakefield winds up his graphic but somewhat sensational account by describing another religious service, which may appropriately be inserted here.",
    "A modern artist would have difficulty in doing such accurate work.",
    "with the particular purposes of the agency involved. The Commission recognizes that this is a controversial area",
    "Oswald's Marine training in marksmanship, his other rifle experience and his established familiarity with this particular weapon",
    "According to O'Donnell, quote, we had a motorcade wherever we went, end quote.",
    "Dr. Clark, who most closely observed the head wound.",
    "Euins, who was on the southwest corner of Elm and Houston Streets testified that he could not describe the man he saw in the window.",
    "Energy enters the plant, to a small extent.",
    "once you know that you must put the cross hairs on the target and that is all that is necessary.",
    "the fatal consequences whereof might be prevented if the justices of the peace were duly authorized",
    "Speaking on a debate on prison matters, he declared that",
    "he was reported to have fallen away to a shadow.",
    "His disappearance gave color and substance to evil reports already in circulation that the will and conveyance above referred to",
    "Here the tread wheel was in use, there cellular cranks, or hard labor machines.",
    "you tap gently with your heel upon the shoulder of the dromedary to urge her on.",
    "This plan of mine is no attack on the Court;",
    "No nightclubs or bowling alleys, no places of recreation except the trade union dances. I have had enough.",
    "The police asked him whether he could pick out his passenger from the lineup.",
    "During his Presidency, Franklin D. Roosevelt made almost four hundred journeys and traveled more than three hundred fifty thousand miles.",
    "He was seen afterwards smoking and talking with his hosts in their back parlor, and never seen again alive.",
    "long narrow rooms, one 36 feet, six 23 feet, and the eighth 18",
#     "We come to the sermon.",
    "even when the high sheriff had told him there was no possibility of a reprieve, and within a few hours of execution.",
    "but there is a system for the immediate notification of the Secret Service by the confining institution when a subject is released or escapes.",
    "When other pleasures palled he took a theater, and posed as a munificent patron of the dramatic art.",
    "Old exchange rate in addition to his factory salary of approximately equal amount",
    "Hill had both feet on the car and was climbing aboard to assist President and Mrs. Kennedy.",
    "seeing that since the establishment of the Central Criminal Court, Newgate received prisoners for trial from several counties.",
    "then let twenty days pass, and at the end of that time station near the Chaldasan gates a body of four thousand.",
    "While they were in a state of insensibility the murder was committed.",
    "reached the same conclusion as Latona that the prints found on the cartons were those of Lee Harvey Oswald.",
    "These were damnatory facts which well supported the prosecution.",
    "but were the precautions too minute, the vigilance too close to be eluded or overcome?",
    "but his scribe wrote it in the manner customary for the scribes of those days to write of their royal masters.",
    "The inadequacy of the jail was noticed and reported upon again and again by the grand juries of the city of London.",
    "At last, in the twentieth month.",
    "which he kept concealed in a hiding place with a trap door just under his bed.",
    "He married a lady also belonging to the Society of Friends, who brought him a large fortune, which, and his own money, he put into a city firm.",
    "Roger D. Craig, a deputy sheriff of Dallas County.",
    "Other officials, great lawyers, governors of prisons, and chaplains supported this view.",
    "who came from his room ready dressed, a suspicious circumstance, as he was always late in the morning.",
    "is closely reproduced in the life history of existing deer. Or, in other words.",
    "accordingly they committed to him the command of their whole army, and put the keys of their city into his hands.",
    "Mrs. Kennedy chose the hospital in Bethesda for the autopsy because the President had served in the Navy.",
    "From those willing to join in establishing this hoped for period of peace.",
    "Muller, Muller, He's the man, till a diversion was created by the appearance of the gallows, which was received with continuous yells.",
    "Years later, when the archaeologists could readily distinguish the false from the true.",
    "his defense being that he had intended to commit suicide, but that, on the appearance of this officer who had wronged him.",
    "together with a great increase in the payrolls, there has come a substantial rise in the total of industrial profits",
    "After this the sheriffs sent for another rope, but the spectators interfered, and the man was carried back to jail.",
    "and improve the morals of the prisoners, and shall insure the proper measure of punishment to convicted offenders.",
    "drove to the northwest corner of Elm and Houston, and parked approximately 10 feet from the traffic signal.",
    "This is the approximate time he entered the rooming house, according to Earlene Roberts, the housekeeper there.",
    "The criteria in effect prior to November 22, 1963, for determining whether to accept material for the PRS general files",
    "and the deepest anxiety was felt that the crime, if crime there had been, should be brought home to its perpetrator.",
    "but his sporting operations did not prosper, and he became a needy man, always driven to desperate straits for cash.",
    "He was soon afterwards arrested on suspicion, and a search of his lodgings brought to light several garments saturated with blood;",
    "He never reached the cistern, but fell back into the yard, injuring his legs severely.",
    "when he was finally apprehended in the Texas Theatre. Although it is not fully corroborated by others who were present.",
    "and she must have run down the stairs ahead of Oswald and would probably have seen or heard him.",
    "afterwards express a wish to murder the Recorder for having kept them so long in suspense.",
    "nearly indefinitely deferred.",
    "On October 25.",
    "They entered a stone cold room, and were presently joined by the prisoner.",
    "that he could only testify with certainty that the print was less than three days old.",
    "Mrs. Mary Brock, the wife of a mechanic who worked at the station, was there at the time and she saw a white male.",
    "Chapter 7. Lee Harvey Oswald Background and Possible Motives, Part 1.",
    "The arguments he used to justify his use of the alias suggest that Oswald may have come to think that the whole world was becoming involved",
    "the number and names on watches, were carefully removed or obliterated after the goods passed out of his hands.",
    "On the 7th July, 1837.",
    "contracted with sheriffs and conveners to work by the job.",
    "at a distance from the prison.",
    "These principles of homology are essential to a correct interpretation of the facts of morphology.",
    "On one occasion Mrs. Johnson, accompanied by two Secret Service agents, left the room to see Mrs. Kennedy and Mrs. Connally.",
    "which Sir Joshua Jebb told the committee he considered the proper elements of penal discipline.",
    "At the first the boxes were impounded, opened, and found to contain many of O'Connor's effects.",
    "on Brennan's subsequent certain identification of Lee Harvey Oswald as the man he saw fire the rifle.",
    "11. If I am alive and taken prisoner.",
    "yet he could not overcome the strange fascination it had for him, and remained by the side of the corpse till the stretcher came.",
    'I noticed when I went out that the light was on".',
    "He was never satisfied with anything.",
    "and others who were present say that no agent was inebriated or acted improperly.",
    'He was in consequence put out of the protection of their internal law ". Their code was a subject of some curiosity.',
    "Let me retrace my steps, and speak more in detail of the treatment of the condemned in those bloodthirsty and brutally indifferent days.",
    "The original plan called for the President to spend only one day in the State, making whirlwind visits to Dallas, Fort Worth, San Antonio, and Houston.",
    "Mr. Sturges Bourne, Sir James Mackintosh, Sir James Scarlett, and William Wilberforce.",
]

In [12]:
texts = [
    "Find the twin who stole the pearl necklace.",
    "Find the twin who stole the pearl necklace.",
    "Find the twin who stole the pearl necklace.",
    "Cut the pie into large parts.In a large bowl, whisk",
    "Cut the pie into large parts.In a smaller size, cut",
    "Cut the pie into large parts. You will need:.1 teaspoon",
    "Burn peat after the logs give out.How does this work?.When you",
    "Burn peat after the logs give out.For the record, I have a bunch of",
    "Burn peat after the logs give out. You can use a shovel to dig up some.",
    "Plead with the lawyer to drop the lost cause.\"It's a very sad day for the family,\" said",
    "Plead with the lawyer to drop the lost cause.\"It's not something that is going to change the outcome",
    "Plead with the lawyer to drop the lost cause.The judge also dismissed a lawsuit brought by the family of the",
    "A shower of dirt fell from the hot pipes. The water was dripping and the water was dripping.\"Oh, oh, oh,\" the",
    "Open your book to the first page.Now, if you have any questions or concerns about this tutorial, please email me.",
    "She blushed when he gave her a white orchid.\"I can't believe how much you've changed in the last six months,\" he said. \"I",
    "A shower of dirt fell from the hot pipes. The rain fell from the ceiling.The man walked to the window and closed the door.",
    "A shower of dirt fell from the hot pipes. A large hole was left in the floor. The door shut. The fire was soon extinguished.",
    "She blushed when he gave her a white orchid. She was so happy when she picked it up from the ground.\"You can't have this, can",
    "She blushed when he gave her a white orchid. The girl nodded and took a deep breath.\"You're not going to want to see me with you",
    "A zestful food is the hot cross bun.While we're at it, we want to know, why do you have to go to the store and buy a whole bunch of these?.We want",
    "Plead with the lawyer to drop the lost cause.\"I'm not going to let that happen,\" he said. \"I'm going to do my job.\".The case is now on appeal.AlertMe",
    "Fine soap saves tender skin. Fine soap saves tender skin. Fine soap saves tender skin. Fine soap saves tender skin. Fine soap saves tender skin. Fie soap tender.",
    "The lure is used to catch trout and flounder. But the lure also offers a great way to keep the fish away from the mouth of a stream.\"You have trout that's",
    "The lure is used to catch trout and flounder. They are a great feeder for the big game. The catch is a little slow but it catches at least 5-8 trout per day",
    "The lure is used to catch trout and flounder.\"A lot of our catch is coming from the south,\" said Dave Lippie, the director of the Fish and Wildlife Department",
    "A zestful food is the hot cross bun. It's a quick, easy, tasty, gluten-free cross bun with a side of chocolate chips.I've tried several variations of these, but there's",
    "A zestful food is the hot cross bun. It's a great way to start a new day.This post may contain links to Amazon or other partners; your purchases via these links can benefit Serious Eats",
    "An abrupt start does not win the prize. The winner will receive a prize of $100.00.If the top 3 players of each group receive the same prize, the winner will receive $20.00 in each group.",
    "An abrupt start does not win the prize. The challenge is to get up to speed with the game and keep your eyes on the prize.The prize is $500.00. The prize is available online.The prize is $",
    "An abrupt start does not win the prize.The winners of this tournament, are:.The winner of the first round of the tournament will receive:.The prize money, which will be distributed by the winner, to the winner",
    "Open your book to the first page.And then go back to the previous page.And then go back to the next page.But let's go back to the first page.And then go back to the next page.And then go back to the next page.",
    "See the player scoot to third base.After the game, I'm going to go back to my old blog and post some more pics of the rest of my game.It's going to be a fun game to play. I hope that you like it!.Advertisements",
    "See the player scoot to third base.This is when you'll be able to take a hit.If you're not able to hit a player, they'll likely take you down.The first thing you'll see is that the base runner can't swing the bat",
    "The long journey home took a year. It was a long journey. It was a long journey. It was a long journey. It was a long journey. It was a long journey. It was a long journey. It was a long journey. It was a long journey",
    "See the player scoot to third base. In order to reach the ball, you must first hit the field goal.On the next pitch, you must hit the field goal.In order to reach the ball, you must first hit the field goal.In order to",
    "Go now and come here later.You can also play the game from your couch or in the living room. You can also play with friends through the side-scrolling game.The game is currently in Early Access.You can sign up for beta now.",
    "The long journey home took a year.A little over a year ago, I got a call from a friend of mine who was working on the project. He was working on a project about the future of the world. He said he was writing a book about the",
    "The long journey home took a year. A few weeks after I left the hospital, my family and I had finally found a place to live. I was in the hospital for two weeks, and I was not ready for the long road back home. I had a lot of",
    "A brown leather bag hung from its strap.\"I'm going to have a party today,\" he said.She had a big grin on her face.\"We're going to have a party,\" he said, in a low voice.\"I'm going to have a party,\" she said.He looked up at her",
    "She has a smart way of wearing clothes. I get to go out to dinner with her, and she's like, \"I want to go to the gym and sit in a chair with you.\" And I don't. I don't want to. I'm not a physical person. I'm not a social person.",
    "A brown leather bag hung from its strap.It was the only one of its kind in the world.It was a piece of metal.Its head was metal.It had a pair of metal eyes.The first one was a black one.The second one was blue.It had a black nose.",
    "The slang word for raw whiskey is booze.The word for raw whiskey is \"raw\" and \"raw\" is not the same thing.The word for raw whiskey is \"clean\" and \"clean\" isn't the same thing.The word for raw whiskey is \"clean\" and \"clean\" isn't the same",
    "The slang word for raw whiskey is booze.The \"sucker in the middle\" is a slang term for someone who has a bad taste in whiskey.The \"sucking\" is a slang term for someone who loves to suck.The \"sucking\" is a slang term for someone who likes to suck.",
    "She has a smart way of wearing clothes. If you're not wearing the right clothes, you're not going to be able to do the job. You have to have a lot of energy. But if you're not wearing the right clothes, you're not going to be able to do the job.\"If",
    "Keep the hatch tight and the watch constant.1.5.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.",
    "A brown leather bag hung from its strap. \"That's the only way I'll ever be able to get out of here,\" he said.\"We're going to be here,\" said the woman.\"We're going to take care of you,\" she said, \"and we're going to keep you safe. If you're not, don't",
    "The sink is the thing in which we pile dishes. The whole thing is there. The sink is what we put on a table.\".There are other items like that, too: \"The sink is the thing in which we pile dishes. The whole thing is there. The sink is what we put on a table.\"",
    "The harder he tried the less he got done.\"I'm not a bad guy,\" he says. \"I'm just not a good guy.\".He was also a good guy.He was a good guy.He was a good guy.He was a good guy.He was a good guy.He was a good guy.He was a good guy.He was a good guy.He was a good guy.",
    "She has a smart way of wearing clothes. She wears a skirt, and her hair is a little wavy, which is pretty much the only way she dresses. She does a little of that with her hair, which is pretty much what she does on a daily basis.There are women who can be pretty much",
    "The slang word for raw whiskey is booze.Wishful thinking.Wishful thinking is the act of thinking, or at least thinking about something, that you don't really understand.Wishful thinking is a type of thought-processing that is often associated with mental illness.Wishful thinking is a",
    "He picked up the dice for a second roll. I rolled a half-and-half, half-and-half. I was sure I'd made a lot of mistakes, but I didn't think I'd make the wrong one. I rolled a half-and-a-half, but I didn't know how to use it. I rolled a half-and-a-half, but I didn't know how to use it.",
    "He picked up the dice for a second roll. The first two rolls were the same, but he had a more precise roll for the second one. The third roll had been the same, but he had a different roll for the third one.\"What?\".\"You're going to have to roll to get to the end of the page.\".\"Don't worry about it. I'll get to it.\"",
    "Find the twin who stole the pearl necklace. After you finish the quest 'A New Hope', you can use the necklace to obtain the quest 'A New Hope' which is a quest that requires you to meet the twins.You are given the necklace by the twins on your way to the town of Riften. It gives you the quest 'The Lost Pearl'.The twins",
    "A pink shell was found on the sandy beach.The shells were found at a large beach in the village of Dauphin, near the border with Senegal.The shell was found by a fisherman off the coast of the village of Dauphin, around 50km from the border with Senegal.The fisherman, who did not want to be named, said the shell was found in the",
    "He picked up the dice for a second roll.The game was about to end, and he tried to think of something to do with it. He thought about a couple of other things, but didn't know if he was going to get anything. And he asked himself, \"Is this the end of the card game?\".\"I guess I'd just like to play it again.\".\"It's just like playing the",
    "Find the twin who stole the pearl necklace. You'll need to find the other one, and the rest of the treasure.The next time you encounter a fairy, you'll get an option to take the pearl necklace from them. Once you've got the pearl necklace, you can return to the princess.The princess can return your pearl necklace to her, but it's not necessary",
    "Raise the sail and steer the ship northward.The ship's speed is 30 knots, but the boat has a maximum speed of 25 knots.\"We had an opportunity to test our skills, and we were able to get by with a bit of luck,\" said Scott.\"The wind was so strong that we could not see the ships coming, but it was a very good test for us to get a feel for the ship and to",
    "It is hard to erase blue or red ink.I am not saying that I am not a red hat. I am saying that I am not a red hat.I am not saying that I am not a red hat.I am not saying that I am not a red hat.I am not saying that I am not a red hat.I am not saying that I am not a red hat.I am not saying that I am not a red hat.I am not saying that I am not a red hat.",
    "A pink shell was found on the sandy beach.The first person to come forward with a description of the suspect was a 32-year-old man, age 27, of North Charleston, South Carolina.The suspect was last seen wearing a black sweater and white pants.The woman who responded to the call told police that she was in an upstairs bedroom with her husband, a man who was with",
    "A pink shell was found on the sandy beach.The mother of four told police the toddler was playing in the water with a friend. She said she thought the child was being sexually abused and was worried it could be her baby.Her daughter gave police a photo of the child and said she had found the child's body in the water.Police said the baby's body was found in the sand",
    "Raise the sail and steer the ship northward.\"This is a tricky task. You have to make sure that the ship is in good condition,\" he said. \"What you need to do is make sure that you get the right materials that will allow you to steer the ship. And that's what we've found to be the most difficult part.\".There are two ways to do it. The first is to use a sailboat to pull it",
    "Find the twin who stole the pearl necklace.Find the pair who stole the pearl necklace. Find the twin who stole the pearl necklace. Find the pair who stole the pearl necklace. Find the twin who stole the pearl necklace. Find the twin who stole the pearl necklace. Find the twin who stole the pearl necklace. Find the twin who stole the pearl necklace. Find the twin who stole the",
    "Raise the sail and steer the ship northward.The ship will take a short trip to the south, but will be sailing near the north coast of England. The sails will be ready for the trip by the time the voyage is over.The ship will be sailing at a rate of 12 knots per hour.All the sails must be raised before the voyage is over and the ship will be sailing at the rate of 10 knots per hour.",
    "Fasten two pins on each side.You can use either the front or back of the keyboard.Insert the keyboard in the front of the case.When you press the key, the case will open.There is an extra button if you click on it.If you change the case, it will be moved and you can use it again.The case will open again.If you press the key again, the case will open.There is a second button if you click on it.",
    "A vent near the edge brought in fresh air.\"I'm sure you'll be able to see the wind on the horizon for a while, but I'm afraid I can't.\".The wind was blowing from the west, making it windy and dry.\"I see,\" I said.\"It's really cold there, isn't it?\" he asked.\"Yeah, it's like we're in a cold place,\" I said.\"Well, you could be able to see it from a few hundred metres away.\".\"I'm sure you can, but I'm",
    "The bank pressed for payment of the debt. The bank refused. On December 6, the court ordered the bank to pay the debt.A few days later, on December 11, I was in a bar with a friend. He asked if I wanted to come to the bar to talk with him, and I said yes.Later that day, I went to a local bar for a drink. When I got there, I started walking around the bar, and then got out of the car.I didn't know what",
    "Fasten two pins on each side.Use a light, small, and sturdy piece of metal. Use a screwdriver to pull the wire from the pins.Remove the pins from the pins and put the wire back in.Put the wire back in and turn it on.Continue to turn the light on.The wiring should look like this.Now you'll be able to use the LED for a quick light.Step 2: Install the Sensor.The sensor is installed on the back of the Arduino",
    "Fasten two pins on each side. Put a small screwdriver or similar into each of your pins.Insert the new pin into the same hole of the old pin.Insert the new pin into the same hole of the old pin.Insert the new pin into the same hole of the old pin.Insert the new pin into the same hole of the old pin.Insert the new pin into the same hole of the old pin.Insert the new pin into the same hole of the old pin.Insert the",
    "The harder he tried the less he got done.\"I'm just trying to get it done. But because I'm trying to do it, it's hard to do it because you're not going to be able to do it. It's not going to be easy.\".The only time he could get past the first 15 minutes was when he was injured in the second half of last season's Champions League semi-final against Juventus.The Argentine striker was also one of the few players to get",
    "The bank pressed for payment of the debt.\"If you can't pay the debt, then that is not your responsibility,\" said Mr. Kowalski, one of the bank's former employees. \"If it's not paid, then you are doing something wrong.\".Mr. Kowalski, who is now a senior vice president at Bank of America, said he will not be returning to the company to pay off his mortgage.Mr. Kowalski said his company's restructuring plan included the sale",
    "Plead with the lawyer to drop the lost cause.\"This is a long-term problem,\" he told us. \"We have to get the government to put in place a mechanism to address it. It has to be done quickly.\".The government has already put in place an online system to track lost and stolen phones and a mobile payment system to help keep track of the phone numbers of the people who lost them.The phone numbers of those who lost their phones are",
    "The odor of spring makes young hearts jump.One of the most common reasons for young hearts to fall is anemia. This is a condition that happens when the tissues of the heart are being damaged.How should you treat your young hearts?.It's important to note that this is just a general guideline. The best way to treat your young hearts is to avoid taking medicines that cause the problem.Learn More.Read More: How to Treat Young Hearts",
    "A six comes up more often than a ten. The top three in terms of overall score are:.Growth Rate: 5.8%.Performance Rate: 5.6%.Speed: 5.2%.Strength: 5.1%.Strength: 5.1%.Strength: 5.1%.Strength: 5.1%.Strength: 5.0%.Strength: 5.0%.Strength: 4.9%.Strength: 3.5%.Strength: 2.9%.Strength: 2.9%.Strength: 2.9%.Strength: 2.9%.Strength: 2.9%.Strength: 2.8%.Strength: 2.8%.Strength: 2.8%.Strength: 2.8%.Strength: 2.7%.Strength: 2.7%.Strength: 2.6%.Strength",
    "The harder he tried the less he got done. At a local barbecue last week, a large group of people were protesting over the loss of a landmark from the city's historic South Side.When the fire started, the building's owner, a former police officer, called the fire department.\"I've always been a police officer,\" he said. \"But I've never seen a fire that big for a police department, and I've never seen a fire that big for the city of Baltimore.\"",
    "Look in the corner to find the tan shirt.\"What's that?\" she asks.\"It's the same shirt,\" he replies.He shrugs. \"It's the same shirt.\".She smiles at him. \"I'll see you later,\" she asks.\"I'm sure,\" he says. \"I'll see you later.\".They walk through the door. The door swings open and the two of them walk into the living room.\"Hey, what's up?\" she asks as the two of them walk into the living room.\"I'm not going to tell you,\" he says. \"I'm not going to tell",
    "Look in the corner to find the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to see the tan shirt.Go outside to",
    "Drop the ashes on the worn old rug.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.A small, white flower.",
    "Plead with the lawyer to drop the lost cause.The court ruling, which covers the entire case, was filed by the ACLU, which argued that the government had been able to get a warrant for the search of a person's cell phone, and that the government must show that a warrant could be obtained without a reasonable search.The court also ruled that the government must show that a warrant could be obtained without a reasonable search if there was a reasonable expectation that the person's \"",
    "It is hard to erase blue or red ink. You need to be able to see what's going on, you need to be able to see what's going on.\".\"You know, I think we're going to be a little bit more careful with what we do in practice, but I think we're going to be more careful with our approach in the offseason.\".\"I think the next step is to get more comfortable with what our guys are doing and how we're going to execute. I think our defense will be a little bit better than what they were last year.",
    "The sink is the thing in which we pile dishes. And I'm not sure what it is, because I think it's a matter of taste, and it's a matter of taste. And that is not what I want.\".As an American, I am not a fan of the idea of a sink that is so deep that it will sink a person. I am not a fan of the idea of a sink that is so deep that it will sink a person.So what does that mean?.It means that the sink is not really a place that you can get out of. It's a place that is where you can get out of",
    "It is hard to erase blue or red ink. The only way to do this is by using a simple, yet effective, method of purification.Pour ink into a glass or a glass bottle and stir until dissolved.Remove the glass or bottle and carefully peel the blue ink off the surface.Now, remove the glass or bottle and carefully peel the red ink off the surface.Repeat this process until the blue ink is completely absorbed.Once the blue ink has completely absorbed, it will be useless.Once it has absorbed, it is",
    "The purple tie was ten years old.\"I've always been a big fan of the big red one. It was a great piece of art, and it's done in a way that's done well. It's done by a professional artist. We were in a big art show in the early 90s and a lot of the guys that did it were very talented, and they were like, 'OK, what the hell are you doing?' I was like, 'You want to do this?' They said, 'No, no, we're going to do it,' so we went out to a lot of shows and they were like, 'Yeah, you have to do it",
    "Go now and come here later.A.D.We're going to build this library for you in our own time. We're going to build it for you to use in your own projects, and we're going to build it for you to use as a starting point for your own websites, and we're going to build it for you to use as a starting point for your own websites, and we're going to build it for you to use as a starting point for your own websites, and we're going to build it for you to use as a starting point for your own websites, and",
    "A vent near the edge brought in fresh air.\"The next morning, we were just taking a bath,\" he said. \"I didn't notice it was cold, but I didn't know it was cold. I was just kind of sitting there and listening to the air.\".The temperature rose to 32 degrees in the morning and then dropped to 37 degrees in the evening, he said.\"It was like a big, black hole, like a big, huge hole,\" he said.\"People were all shaking and they were crying, and they were all scared, because they weren't able to breathe.\"",
    "The bank pressed for payment of the debt.\"The bank may or may not have been aware of the situation and may not have been able to provide information,\" said a document filed with the federal government.The document, which was provided to CBC News by a former employee, makes clear that the bank had to pay an estimated $4 million in interest on the debt.\"We are unable to provide any further information on the situation or the situation of the bank, but we have been advised of the situation and have made",
    "The odor of spring makes young hearts jump.Fruit, eggshells, and leaves look so fresh and juicy. The smell is so strong it makes you think of a big tree.Creamy fruits, such as strawberries, grapefruit, and apples, are so juicy that they are so soft.A nice smell is a good sign that the soil is ripe.It is a good sign that the soil is ripe.A great sign is that the soil is ripe.A good sign is that the soil is ripe.A good sign is that the soil is ripe.A great sign is that the soil is ripe.A good sign is that the",
    "The purple tie was ten years old. It was tied to a belt on the back of the neck. It was pulled with a heavy, hard twig and a small twig. The twig was held in place by the belt and the tie was pulled with a twig. The twig was held in place by the belt and the tie was pulled with a twig. The twig was held in place by the belt and the tie was pulled with a twig. The twig was held in place by the belt and the tie was pulled with a twig. The twig was held in place by the belt and the tie was pulled with a twig. The twig",
    "Go now and come here later.It's a nice place to come down and meet a couple of guys. And they'll tell you that's what they're going to do.We are going to talk about how much we love playing this game and how much we love playing this game, and then we're going to talk about how much we love playing this game and how much we love playing this game. And then we're going to talk about how much we love playing this game and how much we love playing this game. And then we're going to talk about how much we love playing this game",
    "The purple tie was ten years old. He was 6'4\" and had a pair of dark blue jeans. He had a pair of dark blue pants. He was wearing a black trench coat. He had a pair of dark blue jeans. He had dark blue socks and a dark blue hat. He had a dark blue shirt with a white belt. He had dark blue pants. He had a dark blue hat. He had a dark blue shirt with a white belt. He had a dark blue hat with a white belt. He had a black hat with a white belt. He had a black hat with a white belt. He had a black hat with a white belt. He had a black",
    "A vent near the edge brought in fresh air.\"We had to find another way to get to the other side of the building. We found a couple of people and then we found the last guy who was in the building. It was very important to find him,\" said Kiyotema.The other two men were shot in the head and were taken to a local hospital.\"They were shot without any injuries and they were taken to the hospital.\".The victims were taken to the hospital in a critical condition and are expected to be released later today.Police said the shooting was a \"very",
    "The bill was paid every third week. The average cost was $10.7 million.The bill also included a $2.3 million increase in state funding for the Public Safety Department.A separate bill, the Public Safety and Public Safety Reform Act, passed Thursday night, is being considered by the Senate Judiciary Committee.The bill would have allowed state law enforcement to use the funds for \"community policing\" or \"community partnerships\" without having to pay for them.The bill is expected to be signed into law by Gov. Scott Walker in the spring.",
    "Open your book to the first page.And then click the \"Add to Book\" button.Now you can use your Kindle's \"Create New Book\" button to create a new book in the Kindle.Make sure to create a new book when you click the \"Save\" button.If you're having trouble with the Kindle, you can try downloading the Kindle App from the App Store.The Kindle App is here!.If you have any questions about Kindle, please ask in the comments below.Now, if you have any questions about this app, please contact me at: mike.kaufman@gmail.com or at: mike.kaufman@gmail.com.",
    "Fill your pack with bright trinkets for the poor.6. Get the best quality yarn at a good price.7. Get the best quality yarn at a good price.8. Get the best quality yarn at a good price.9. Get the best quality yarn at a good price.10. Get the best quality yarn at a good price.11. Get the best quality yarn at a good price.12. Get the best quality yarn at a good price.13. Get the best quality yarn at a good price.14. Get the best quality yarn at a good price.15. Get the best quality yarn at a good price.16. Get the best quality yarn at a good price",
    "Drop the ashes on the worn old rug. The corpse is now a dead one.The grave marker is now the same as the one on the grave marker.The final set of items for the quest \"Staying Alive\" has been added.The quest \"Inquisitor's Way\" now requires all the items from the \"Inquisitor's Way\" questline.The quest \"Staying Alive\" has been added.The quest \"Staying Alive\" has been added.The quest \"Inquisitor's Way\" now requires all the items from the \"Inquisitor's Way\" questline.The quest \"Inquisitor's Way\" has been added.The quest \"Inquisitor's Way\" has been added",
    "The sink is the thing in which we pile dishes. The kitchen is the thing which can be put on the sink. The dining room is the thing which, if you're not in it, can be put on the table. We're all in this together.One thing I've noticed about the way we all work together is that we are all doing what we know is right. I'm not saying that we're always doing the right thing. Sometimes when we're working together, we're doing what we know is right. Sometimes when we're not working together, we're doing what we know is right. I think a lot of the time, it's",
    "The odor of spring makes young hearts jump.\"I'm a little bit ashamed to admit that I was just trying to keep up with the pace,\" he said. \"I was just trying to keep up with the pace and keep up with the kids. They were really excited. They were just getting ready to go to school for the first time.\"I was just doing my best and was just trying to get my body right.\".The University of Miami School of Medicine is the only school in Florida to offer a \"Dietary Plan\" for kids to help them lose weight.The plan includes a daily activity plan with nutrition, exercise and weight loss.\"I'm thankful that my body was",
    "Keep the hatch tight and the watch constant. This means that once you get the hang of it, you'll be ready for the next step. You'll want to work with the watch to make sure that the light is on and it's ready to go when you get out of the car.When you're ready to get out of the car, you'll want to place the watch on a flat surface (not on the ground) and place it on the floor. This will make it easier to keep your watch from falling off the watch.To keep it upright, you'll want to place the watch in the middle of the house. It's best to place the watch in the middle of the house and to hold it in place.Once",
    "Look in the corner to find the tan shirt. You can find the shirt on the side of the car. There is also a red badge on the front of the shirt. This is where you can tell the driver is driving erratically. If the driver is driving erratically, you will see the white badge.The red badge indicates that the vehicle is in a traffic stop. If the driver is driving erratically, you will see the yellow badge.If the driver is driving erratically, you will see the red badge. If you see the yellow badge, you will see the yellow spot on the back of the vehicle where the red spot is. This is where you can tell the driver is driving",
    "He lent his coat to the tall gaunt stranger. He asked him if he had seen the woman's face.\"I don't know,\" Merel said. \"She's not a woman. Some of the women look like women. I don't know if she's a man or a woman. She's a man. She's a woman. She's a man.\".\"I don't know, I don't know,\" the stranger said. \"I don't know if I've ever seen her face before.\".\"Well, I don't know what to say, I don't know what to say,\" Merel said. \"I don't know what to say.\".\"I don't know, I don't know,\" the stranger said. \"I don't know if she's a man or a woman. She's a man. She's a woman. She's a man.\".Merel looked at the man. He was a woman",
    "Drop the ashes on the worn old rug.The next day, the family moved to a new house on the hill. They found some old books and a table.\"We were sitting there, and he said, 'There's this book that you gave me, and it's a book you're supposed to read,'\" said one of the girls. \"And we went and read it.\".As the family moved, the house became more and more haunted.\"They would say, 'You know, if you're looking at these books, you're going to get a lot of things wrong,' \" said the girl. \"But I'm not going to get these things wrong, even if it's a book that I think I read.\".The family took their children to a local haunted house.",
    "Fill your pack with bright trinkets for the poor.The new \"Crimson Pendant\" is designed to keep your \"Crimson Pendant\" in the hand while you are using it. This means that it can also be used with a pocketknife or an old-fashioned pocket knife. The \"Crimson Pendant\" is designed to be worn on your hands while you are using it. This means that it can also be used with a pocketknife or an old-fashioned pocket knife.The new \"Crimson Pendant\" is designed to keep your \"Crimson Pendant\" in the hand while you are using it. This means that it can also be used with a pocketknife or an old-fashioned pocket knife.This is the original \"",
    "The vane on top of the pole revolved in the wind. The wind was so strong that the wind had to be stopped to keep the pole from being pulled in to the air. This was done by a simple motion of the vane. The wind blew the pole up and down as it moved. The wind got in and out of the pole as it moved. The wind then got in and out of the pole as it moved. The wind then got out of the pole as it moved. The wind then got out of the pole as it moved. The wind then got out of the pole as it moved. The wind then got out of the pole as it moved. The wind then got out of the pole as it moved. The wind then got out of the pole as it moved",
    "The horse balked and threw the tall rider.\"You're going to have to get up,\" he said.\"I'm going to have to get up,\" the horse said.\"We're going to have to go,\" the horse said.The horse and rider were both able to escape, but the rider was unable to walk.\"She just went through the door,\" said the horse, who was also able to walk.\"The horse can't walk. The horse is going to have to get up,\" said the horse.The horse, who is not named, was able to walk but has not been able to drive.\"We're going to have to get up. We're going to have to get up,\" the horse said.The horse was able to walk but has not been able to drive.\"We're going to",
    "A six comes up more often than a ten.The following is a list of the best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best of the best.The best",
    "Fill your pack with bright trinkets for the poor.How to make the Best of Your Life.I'm not saying you can't make a great of your life by putting a lot of effort into getting the right things done. If you do, you'll be able to save time and money. If you're not sure what you want, you can ask the person who's working on your project.If you're not sure what you're looking for, ask the person you're working with for advice.If you're not sure what you're looking for, ask the person who's working on your project.If you're not sure what you're looking for, ask the person who's working on your project.Don't forget to check out my other blog",
    "The vane on top of the pole revolved in the wind. The wind was fierce and the wind was heavy. The pole was moving at a steady speed. The wind was strong and the wind was heavy. We were all on the ground and the wind was strong and the wind was heavy. The wind was strong and the wind was heavy. I was in the middle of the pole. I felt very good. There were several people on the pole and I was in the middle of the pole. I was very happy. I felt very happy.\"I was in the middle of the pole. I felt very happy. There were several people on the pole and I was in the middle of the pole. I felt very happy. There were several people on the pole and I was",
    "A speedy man can beat this track mark.I've seen this guy over and over again, and he's never looked like he's running out of gas. He's never looked like he's getting to the end of his game, and he's never looked like he's slowing down.I don't think he should be out of his prime, but if he was, he could be a lock for the starting job.There's a lot of talk about the '90s. It's a time when the '90s were a lot more than just the '80s and '90s, and the '90s were a time when the '90s were about more than just the '80s.There's a lot of talk about the '90s. It's a time when the '90s were a lot more than just the '80s and '90s, and the '90s were a time when the '90",
    "Keep the hatch tight and the watch constant.The following diagram shows the control panel of the watch. The outer portion is the watch face, while the inner portion is a watch face and the right hand corner contains an indicator. The outer portion is to the right of the watch face.Rear portion of the watch face. See diagram below.The watch face is a white gold plated watch face with a black finish. The watch face is worn on both hands. It has a blue dial and a white dial with an orange dial. Like the watch face, the watch faces are of a larger size than the dial. The watch face is made of stainless steel and is made of a silver color.The watch face is a dark",
    "A speedy man can beat this track mark.This is a really good one.I don't think I've ever seen a runner that fast in this race.This is a real treat to watch.I've been running this race for 10 years. I've never seen anyone that fast in this race.The one thing I don't like about this track is the way the track is set up.This is a little more of a step up than what I expected.I'm not going to go out of my way to say it's a bad track, but it's definitely a step up.This is a great track.I've never seen a runner that fast in this race.I'm not going to say it's a bad track, but it's definitely a step up.This is a really nice track.I'm not going to say it's a bad track",
    "The vane on top of the pole revolved in the wind. It was the only thing that could do it. It was the only thing that could move the wind. It was only the thing that could change the course of the wind. And it was the only thing that could change the course of the wind. And it was the only thing that could change the course of the wind. And it was the only thing that could change the course of the wind.The vane on top of the pole revolved in the wind. It was the only thing that could do it. It was the only thing that could move the wind. It was the only thing that could change the course of the wind. And it was the only thing that could change the course of the wind.",
    "A six comes up more often than a ten.8. No. 9.The \"no\" is probably the most common word in the dictionary. It's easy to remember. It's not that hard to remember.9. No. 10.The \"no\" is probably the most common word in the dictionary. It's easy to remember. It's not that hard to remember.10. No. 11.There's always the \"no\" in the dictionary. It's easy to remember. It's not that hard to remember.11. No. 12.There's always the \"no\" in the dictionary. It's easy to remember. It's not that hard to remember.12. No. 13.There's always the \"no\" in the dictionary. It's easy to remember. It's not that hard to remember.13. No. 14.There's always the \"no\" in the dictionary. It's easy to",
    "The couch cover and hall drapes were blue. The ceiling was white. The walls were painted in white.\"Hello, you're awake.\".\"Oh yeah, I'm awake.\".\"I'm awake.\".\"I'm awake. Now, you're awake.\".\"I'm awake.\".\"I'm awake.\".\"I'm awake. Now, you're asleep. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\".\"I'm awake. I'm awake. Now, you're awake.\"",
    "Either mud or dust are found at all times.The reason for the lack of dust is that at some point, it is necessary to remove the dirt.Mud is not the only thing you need to remove. A good technique for cleaning is to make sure that you have a clean cloth.You can get clean cloths by using a toothbrush or a toothbrush sponge.If you have mud on your hands, or dirt on your feet, be sure to clean it immediately.If you have dust on your feet, be sure to clean it immediately.If you have mud on your hands, be sure to clean it immediately.If you have mud on your hands, be sure to clean it immediately.If you have mud on your hands, be sure to clean it immediately.If you have mud on",
    "He lent his coat to the tall gaunt stranger. \"I'm sorry I couldn't see you, but I was at the door.\".\"I'm sorry I couldn't see you. I've seen you, and I've seen you so many times. You're my friend. You're my friend. You know what I think? I'm going to do anything for you.\".\"The only thing that's going to stop me is the one you've been with.\".\"There's nothing to stop me. I'm just a stranger, so I won't be able to see you.\".\"You're going to be my friend. I'm going to do anything for you. I'm going to do anything for you.\".\"You're going to be my friend. I'm going to do anything for you. I'm going to do anything for you.\".\"You're going to be my friend. I'm going to do anything for you. I'm",
    "He lent his coat to the tall gaunt stranger. \"I'm going to be careful, darling,\" he said, but the stranger shook his head. \"You can't hurt me. I'm not afraid.\".\"I'm not afraid.\".\"I'm not afraid,\" said the stranger.\"I'm not afraid,\" said the man, as he looked at the stranger, and the stranger shook his head.\"I'm not afraid,\" said the stranger.\"I'm not afraid,\" said the man, as he looked at the stranger, and the stranger shook his head.\"I'm not afraid,\" said the stranger.\"I'm not afraid,\" said the man, as he looked at the stranger, and the stranger shook his head.\"I'm not afraid,\" said the stranger.\"I'm not afraid,\" said the man, as he looked at the stranger, and the stranger shook his head.",
    "Fine soap saves tender skin.Soak your skin in hot water for 7 minutes, or until it starts to sizzle.Boil your skin in hot water for 3 minutes, or until it starts to sizzle.Remove the skin from the heat by lifting it off the skin, and then gently using a long, flat spoon to remove excess skin.Remove the skin from the heat by lifting it off the skin, and then gently using a long, flat spoon to remove excess skin.Remove the skin from the heat by lifting it off the skin, and then gently using a long, flat spoon to remove excess skin.Remove the skin from the heat by lifting it off the skin, and then gently using a long, flat spoon to remove excess skin.Remove the skin from the heat by lifting it",
    "The horse balked and threw the tall rider. He tried to ride away, but he was too slow to catch up. The horse's body started to shake, and he was afraid to come back.The horse looked back at the horse and said, \"You've got to keep your head up and keep your hands up. I'll take you back to the grass.\".The horse jumped back, but he was too slow to get back up.The horse went on with the rest of the day, and I never saw a horse that had any fear of dying.I saw a horse of the same age and of the same race, and I never saw a horse that had any fear of death.I remember the horse's name, and, I believe, that was his name, and that we had a good discussion. He said it was an old horse, and he said it was a",
    "Pack the records in a neat thin case.And then you're done.For now, we're going to have to do a bit of experimenting on how to make these different types of records. The first step is to make a record that doesn't contain the data in the main record.We'll start with a typical record, with a single text entry.Text entries are the things that will be stored in a record. They're the things that are written onto the record.Next, we'll add the text entry to the main record.This is where we're going to put the record. It will look something like this.This is the string that we want to store the text entry.We're going to do this by using the formatter.This is where we're going to put the text entry. It's going",
    "The big red apple fell to the ground.\"Ouch!\".\"No!\".\"What's wrong?\".\"Nothing. If you can't stand it, you can't get your hands on it.\".\"So you don't want to, you know?\".\"No. I don't want to, I don't want to.\".\"What is it?\".\"You don't have the guts to say it. I don't want to. But I do want to.\".\"You'll do it if you're a little bit more careful.\".\"Yes, I'll.\".\"Oh, you'll do it. You're not going to get angry if you don't. I'll have you, and I'll have you all right.\".\"You're not going to get mad if you don't. I'm not going to be angry if you don't.\".\"O-okay.\".\"The apple is going to fall. I'll eat it.\".\"I'll eat it, and I'll eat it. I'll be a little bit more cautious, and I'll look for another apple. O-okay.\"",
    "After the dance, they went straight home.\"I don't know,\" she said. \"I'm tired of it. I don't know what to do.\".She said the two men were \"unfair\". She said they approached the house because the police were not there.\"They were very rude and started harassing us,\" she said. \"They say they don't like us because we are black. They don't like us because we are black. They do not like us because we are black.\".The couple were arrested and charged with \"disorderly conduct\".A police spokesman said: \"The complaint was lodged on behalf of the couple.\"The complainant has been released pending further enquiries.\"The complaint was filed on behalf of the complainant.\"Officers are currently investigating the incident.\"",
    "Either mud or dust are found at all times.The first thing to do is to check to be sure you're not getting any dirt on your nails. If you're taking your nails to a salon, you should check to be sure they're covered in dirt. If they are, try to get a clean one and use it to polish your nails and make sure they're covered in dirt.If your nails are covered in mud, try to remove it as soon as possible. If they're covered in dust, try to brush them off with a damp cloth.If you're taking your nails to a salon, try to check to be sure they're covered in dirt. If they are, try to get a clean one and use it to polish your nails and make sure they're covered in dirt. If you're taking your nails to a salon, try to check",
    "The lamp shone with a steady green flame.\"The light was so bright, so bright, that I felt it was the same as my heart. And I had to turn it on. It was very bright, so bright, and I was very sad. I wanted to get out of here.\".She had to turn on the light. She was trying to make out the words.\"I'm not sure where I am. I'm on the edge of the world. I'm not sure where I am. I don't know anything. I'm just trying to find my way. I don't know where I am. I'm just trying to find my way.\".She was trying to make out the words.\"Is it all right?\".\"I don't know. I can't tell you. I can't tell you anymore. I couldn't tell you. I was trapped. I couldn't see. I was trapped. I had no idea what I was doing. I was trapped. I had",
    "A speedy man can beat this track mark.The first thing I noticed when I first started playing the game was that there was no way to do a \"walk\" in the game. I had to use my feet and the ball. All I had to do was hit the ground and walk. I never had to think about how I would do it.I was always a big fan of the \"walk\" in the game. It was just something you did to a man and he would do it. I would do it to myself. I would do it to a man, but when I was with the game, I would do it to myself. The only thing I could do was use my feet. It was just like a chore. I would do it to myself. I would do it to a man, but when I was with the game, I would do it to myself. The only thing I could do was use my feet.I still have that",
    "The horse balked and threw the tall rider.\"It was very difficult to get the horse out of the saddle,\" said David. \"I think the horse was very much in the saddle. It was a very strong horse.\".The horse struggled to get out of the saddle, but David was able to push it out of the saddle. The horse was able to move, but it was a lot of effort to get out of the saddle.The horse was about to jump to the other side of the saddle when David pulled on her hind legs to push it out.\"I think it was very, very difficult to get the horse out of the saddle,\" said David.David quickly got on her back and pulled the horse away from the horse.\"It was very difficult to get the horse out of the saddle,\" said David.The horse was able to move, but it",
    "Fine soap saves tender skin. It helps keep skin moist and dry. It is also a good moisturizer.3. Skin care products that will help you stay hydrated.The skin care companies of your choice will help you with your skin care needs. They will give you the best products to help you stay hydrated.They will also help you with your skin care needs. They will give you the best products to help you stay hydrated.4. Skin care products that will help you stay hydrated.The skin care companies of your choice will help you with your skin care needs. They will give you the best products to help you stay hydrated.They will also help you with your skin care needs. They will give you the best products to help you stay hydrated.5. Skin care products",
    "Pack the records in a neat thin case. I used a small, heavy duty tape measure (I used a 2-inch diameter) and a small, heavy-duty needle. It took about an hour to get all of the records into the metal case. I was able to create a groove in the top of the vinyl that I could fold into a ballpoint pen. I did this to make the vinyl stand up to the pressure of the record. The record was about 5 feet long and about 30\" in diameter. I used a plastic tuck stand to get the record out of the metal case. I then held it in place with a little screwdriver. I used a small, plastic handkerchief to hold the record tightly in place. I also used a small, plastic dremel to hold the record in place. I then used a small, plastic needle to hold the record in place. I used",
    "Either mud or dust are found at all times.\"They're all very familiar with the area and the area is very diverse, and it's very difficult for me to think of anything that would be as common as a mud or dust trail,\" says Tom Hickey, lead researcher on the project.Hickey is not alone in the belief that mud and dust are common. A study in the journal Nature found that nearly all of the commonest mud and dust trails in the United States are in the Northeast.The study, conducted by Hickey, was funded by the National Science Foundation.\"It's a big deal for us to look for these trails that we think are important to the landscape,\" Hickey says. \"We're working with a lot of different people, and this is a really exciting opportunity to do that in a very diverse area.\"",
    "The big red apple fell to the ground.\"What?\".\"You said that the other day. What?\".\"Why not?\".\"Why, I told you, it's not a good idea.\".\"I'm not going to tell you, but it's an awful idea.\".\"Why not? It's a good idea.\".\"Well, that's not all I can say.\".\"Well, I don't know how to say it, but it's your idea.\".\"But I was never going to tell you that, or that I could tell you that.\".\"You know, I don't know. If you can't tell me, then I won't tell you. You have to tell me. You cannot tell me,\" he said.\"I don't want to tell you, I want to tell you. You'll tell me, you can tell me. I won't tell you, I want to tell you. You're going to tell me, and you're going to tell me, and you're going to tell me, and I'm going to tell you.\".\"I know what you mean. I know what you mean",
    "Pack the records in a neat thin case. I used a bit of heavy duty tape and a piece of cloth to keep the records separate from the rest of the case. If you want to keep your records clean in the light of the sun, you can do this with a plastic bag.Use the cloth to keep the records separate from the rest of the case. If you want to keep your records clean in the light of the sun, you can do this with a plastic bag. Clean the records with a damp cloth. I like to use a damp cloth, but it will hold the records.I like to use a damp cloth, but it will hold the records. Use a mirror to make sure your records are sitting right where they should be.You can use a mirror to make sure your records are sitting right where they should be. Wash the records with a damp cloth. I don",
    "The lamp shone with a steady green flame. The light had the feeling of a shining sun.\"Don't you think we'll use that to kill someone?\".\"I think we'll use it to kill someone that's not our enemy. I think we'll use it to kill someone that's not our enemy.\".\"What? You mean we'll use it to kill someone that's not our enemy?\".\"It's just that we're scared, and I don't think we'll use it to kill someone that's not our enemy. I think we'll use it to kill someone that's not our enemy. I think we'll use it to kill someone that's not our enemy.\".\"Why would you use it to kill someone that's not your enemy? You're trying to kill a child, right?\".\"It's not like that, it's just that we're scared. I don't think we'll use it to kill someone that's not our enemy. I think we'll use it to kill",
    "A thin stripe runs down the middle. It's a very soft, non-pale white.The top of the stripe is the same as the white.This is the bottom of the stripe. I have to remove the top of the stripe to make sure it stays flat.The top of the stripe is the same as the white.The bottom of the stripe is the same as the white.The top of the stripe is the same as the white.This is the bottom of the stripe. I have to remove the top of the stripe to make sure it stays flat.The top of the stripe is the same as the white.The bottom of the stripe is the same as the white.This is the bottom of the stripe. I have to remove the top of the stripe to make sure it stays flat.The top of the stripe is the same as the white.The bottom of the stripe is the same as the white.This is the bottom of the stripe.",
    "The price is fair for a good antique clock.The best timepieces in the world.The best timepieces in the world are the ones that are worth the price. You should buy a clock that you can use as your timepiece. If you want to save money, buy a piece that will last you many years.The best watches.The best watches are the ones that will last you many years. You should buy a watch that will last you many years.The best watches are the ones that will last you many years. You should buy a watch that will last you many years.The best watches are the ones that will last you many years. You should buy a watch that will last you many years.The best watches are the ones that will last you many years. You should buy a watch that will last you many years.The best watches are the ones that will last you many years",
    "The price is fair for a good antique clock. It is a good value for money. I would give it 5 stars. The service is great. After you bought the clock and its service, it is worth it.Rated 4 out of 5 by Jeff from Good for the price! The clocks are very well made. The design is very well made. The colors are very nice. I like them.Rated 5 out of 5 by Anonymous from Great Clock I bought this with my older clock. The price is reasonable and the service is very well done. The price is good for the price. The service is good. I would recommend this clock to anyone who is looking for a good antique clock.Rated 5 out of 5 by kathryn from Very Good I bought this to replace a broken clock. I have had this clock for about 2 years, and I have always enjoyed the quality of the service. After a few weeks of use by",
    "Time brings us many changes.The first is that it is possible to remove the first two lines from the first line after a request, so that if you don't like the first line, you can just use the next one. The second is that it is possible to delete the second line from the first line after a request, so that if you don't like the second line, you can just use the next one. The third is that it is possible to add a new line to the first line after a request, so that if you don't like the first line, you can just use the next one.The first two lines are:.I am using the new string to add the next line.The second line is:.I am using the new string to add the next line.The third line is:.I am using the new string to add the next line.The fourth line is:.I am using the new string to add the next line.The fifth line is:.I am using the new",
    "Help the woman get back to her feet.The problem is that we can't do that. We can't even do that, because we're not allowed to do that.I think that's why you're seeing so many people who are saying, \"I don't want to go back to my house, I don't want to go back to my kids, I don't want to go back to my family.\".And then they're saying, \"I don't want to take my kids to school.\".I think that's just the way it is.And I think that's why our culture is really broken up.We're in a really bad situation right now when we're talking about, \"Oh, we have to do more. We have to do more.\".And what we're seeing is that we're going to have to do more.And that's why I think we need to look at the next president.I think we need to look at the next president, because there are so many things at stake.We have to look at the next president.We have to",
    "A pod is what peas always grow in. When a pod is in a pod, the pod is in its own system. It is the only way to tell if a pod is in a pod or not.The pod is in its own system. It is the only way to tell if a pod is in a pod or not. The pod is in a system that has a lot of water and it doesn't have all of the nutrients it needs.The pod is in a system that has a lot of water and it doesn't have all of the nutrients it needs. The pod is in a system that has a lot of water and it has a lot of nutrients that it needs. It has a lot of water, so when it has enough to feed its own pod, it takes them, it builds it, and it has a lot of nutrients.So that's a whole lot of information that I'm not going to get into here.However, that's where it gets interesting.In the book, John A. Kapp (a professor at the University of Pennsylvania) explains that if",
    "The lamp shone with a steady green flame. The light was bright and bright, but it was so bright that I could barely see it. I looked up and saw that it was the only one of its kind.I turned around and saw that the person I was talking to had gone into hiding. I ran to the front of the room, and saw that it was the only one of its kind. I ran to the back of it and saw that it was the only one of its kind.I walked over to the door and opened it. It was a large wooden door, and the lamp was shining with a steady green flame. The light was bright and bright, but it was so bright that I could barely see it. I looked back and saw that it was the only one of its kind. I ran back to the door and saw that it was the only one of its kind.I turned around and saw that the person I was talking to had gone into hiding. I ran to the front of the room, and saw that",
    "We are sure that one war is enough. Let us know of any mistakes.\".The UN Security Council on Wednesday endorsed the idea of a new peace plan by the end of this month. The Security Council said it was committed to the need to end the conflict in Syria by March.The UN Security Council also voted unanimously last week to adopt resolutions calling for a ceasefire in the Syrian civil war, a move that has sparked a diplomatic spat.The council's decision to approve the resolution was confirmed by the Security Council on Wednesday.\"The Security Council is committed to the necessity of a cessation of hostilities and a cessation of hostilities in Syria,\" it said.\"The Security Council recognises the need for an end to all forms of violence in Syria. These include the use of force, intimidation and intimidation, and the use of force against civilians and opposition forces.\"",
    "Help the woman get back to her feet.\"The baby is fine. She's fine. I'm going to take her home and I'll let her go. I'm not going to go into the apartment and I'm not going to go into the kitchen. I'm not going to leave her. I'm going to take her home.\".The man said he wouldn't take her home if she didn't come to him, but he said he was afraid to go out because he had a job.\"We have to make sure that she doesn't come home,\" the man said.The man said he and his wife are also worried about what will happen to their five-month-old daughter, Aida.\"They're going to have to go out and get help,\" he said. \"I'm concerned about Aida's health and she will need a lot of help.\".The man said he is calling the police and is trying to find out what happened.Police say the man told them he was going to take Bibi home after the baby was lost.Police say some of the people who were with",
    "A pod is what peas always grow in. And that's what we're hoping to do with this series.We're going to do the first episode with a very simple concept: It's going to be a pod, and if you want, you can get a cup of peas in it. The pod is going to have a very short life span. It's going to be able to do what you want it to do, and it will be able to do it. That's the magic of this show.So, we're going to have this show where the pod is going to be a series, and there are two stories that we're going to tell. One is a story about a pod. And the other is a story about a pod. And we're going to tell it like this:.And one of the things we want to do with this story is we want to tell stories about how the pod is going to be able to survive. And the way this story will be told is that the reason we're going to have two different pods is because we want to tell stories about how the pod is",
    "The big red apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell to the ground.The white apple fell",
    "Send the stuff in a thick paper bag.I'll give you a few tips.1. Don't use an old-fashioned paper bag.No, don't. The paper bag is not a tool to help you organize your stuff.2. Don't stick the stuff to the ground.The first one is easier. The second one is harder.3. Don't bring it with you.The first one is a good thing. The second one is a bad thing.The third one is a bad thing.4. Don't leave it hanging.The third one is a bad thing.5. Don't leave it in a hurry.The first one is a good thing. The second one is a bad thing.6. Don't leave it in the middle of a meeting.The first one is a good thing. The second one is a bad thing.The third one is a bad thing.The fourth one is a bad thing.7. Don't leave it in the middle of a meeting.The first one is a good thing. The second one is a bad thing.The third one is a bad thing.8. Don't leave it in the middle of a conversation.The first one is a good thing",
    "Act on these orders with great speed.[B]omourne in the court of the Supreme Court of Ontario, and with great speed.[C]omourne, in the court of the Supreme Court of Ontario, and with great speed.[D]omourne in the court of the Supreme Court of Ontario, and with great speed.[E]omourne in the court of the Supreme Court of Ontario, and with great speed.[F]omourne in the court of the Supreme Court of Ontario, and with great speed.[G]omourne in the court of the Supreme Court of Ontario, and with great speed.[H]omourne in the court of the Supreme Court of Ontario, and with great speed.[I]omourne in the court of the Supreme Court of Ontario, and with great speed.[J]omourne in the court of the Supreme Court of Ontario, and with great speed.[K]omourne in the court of the Supreme Court of Ontario, and with great speed.[L]omourne in the court of the Supreme Court of Ontario, and with great speed.[M]omourne in the court of the",
    "The price is fair for a good antique clock. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case. The case is removable and is reversible. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case. The case is made of a durable, high-quality aluminum alloy and is easily protected by a protective coating on the inside of the case.The case is manufactured of solid brass",
    "Help the woman get back to her feet.\"It's a relief,\" she said, smiling. \"It's a very emotional time.\".The woman has been in a coma since the attack. She is expected to survive.The man, who was not identified, was not injured.\"He didn't know anything about the attack,\" says the woman's brother, Jim, who has been with her since the attack. \"He couldn't understand what happened.\".The woman's family said the woman, who was born with a congenital heart defect, had been on medication for about a week.\"She was doing well and she was trying to keep going,\" says her brother. \"She didn't know what to do. She just kept on going.\".The woman's family said she has no issues with her husband.\"She is a very caring, loving wife,\" says the woman's family's attorney, Tom Moore, who is leading the case.The woman, who had been living with her mother in New Mexico, was in a coma for about three weeks. Her sister, who was with her mother",
    "A pod is what peas always grow in. The pods are made of a variety of plant material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as a single stem, a single stalk, and a single stem. The pod is then made of a variety of material, such as",
    "The heart beat strongly and with firm strokes.The blood is very viscous, and the heart is very viscous. The blood is not as viscous as it should be, so it is not as viscous as it should be.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.This is the way in which the brain is kept in equilibrium.",
    "A thin stripe runs down the middle. The center of the stripe is an oval shaped indentation. The inner side of the indentation is a line with a small notch. The outer side of the indentation is a line with a large notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a large notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center of the indentation is a line with a small notch. The center",
    "The heart beat strongly and with firm strokes.\"I've never had a stroke so intense,\" she said. \"It's so satisfying to be able to have this experience with a woman who's been through so much of it.\".\"She's so grateful to be here. She's so grateful to be here with me,\" she said. \"It was just so beautiful.\".The woman's family has been waiting for this moment for more than a week, having traveled to the hospital to help with the recovery process.\"I'm just glad she's going to be able to go through this again,\" said Kristina, who was with the family at the time of the incident. \"I'm just so grateful to be here with her.\".She said the next step is to take her own life.\"It's been so long since I last saw her,\" Kristina said. \"I'm so grateful to be here.\".\"I'm so grateful for the people who have come out of the hospital,\" she said. \"I'm so thankful to the family and the people in the hospital for coming out of the hospital and helping me heal. I'm",
    "They took the axe and the saw to the forest. All the people who did not have money were killed.\"Bastard, what was that?\".\"I was going to tell you that I did not want to die. I have to tell you, people who want to kill me, they will find it hard to believe that they are being treated with such dignity and respect. I have to tell you, I will not let you die. I am a coward, I am a coward who will not allow my life to be taken from me. I will not allow you to be shot. I will not let you be killed. I will not allow you to be raped. I will not let you be killed. I have no choice. I have no choice. I have no choice. I have no choice. I have to die. I have to die. I have to die. I have to die. I have to die. I have to die.\".The crowd of people, in a thicket of men, women, children, the elderly, young, old, old, and young, was shouting.\"He is a coward. He is a coward. He is a coward. He is a coward. He is a coward. He is a coward. He is a coward.",
    "We are sure that one war is enough. If we do not stop one, we will be met with a new one.And that war is the first war of our generation, the war of our children, the war of our grandchildren.It is a war of the future and a war of the past.We are the ones who are to fight for the future.We are the ones that are to be saved.We are the ones who are to be saved by the future.We are the ones who are to be saved by the future.We are the ones who are to be saved by the future.We are the ones who are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones that are to be saved by the future.We are the ones who are to be saved by the future.We are the ones that are",
    "Time brings us many changes.The app is now optimized for Android Wear.When we launched, we said we'd be on Android Wear for the first time. We've been working on the Android Wear platform for a long time, and we're excited to see the mobile version of Android Wear come to the Apple Watch.We're excited to see the mobile version of Android Wear come to the Apple Watch. We're also excited to see the iOS version of Android Wear come to the Android Wear platform.We're also excited to see the Android Wear platform come to the Apple Watch. We're also excited to see the iOS version of Android Wear come to the Android Wear platform.We're going to be releasing Android Wear on the iPhone and iPad.As with all of the updates, the new features we're introducing in our app will be the same as those we've seen in the past.We're going to be releasing Android Wear on the iPhone and iPad.As with all of the updates, the new features we're introducing in our app will",
    "They took the axe and the saw to the forest. \"Now if you don't look after yourself,\" he said, \"you'll have to pay for that.\".\"No,\" said I. \"You're not going to take anything for granted. I'm not going to take anything for granted. I've been through this before, and it's not like I'm going to be able to take it back. I've been through this before. I've been through the whole family. I've been through a whole bunch of things, and I'm going to have to do this as best I can.\".\"No,\" I said. \"You're going to have to pay for it.\".I felt a little like I had been through the entire ordeal, but then I realized I'd taken my axe and saw. I was about to walk out of the house when I heard a knock on the door.\"What's going on?\" I asked.\"You're going to have to pay for this,\" he said. \"But you're going to have to pay for it. We're going to have to pay for this. I'm going to have to pay for this. I'm going to have to pay for this. I'm going to have to pay for this.\".I",
    "The couch cover and hall drapes were blue.\"I've got this,\" said the woman. \"I'm going to start the day with a couple of cocktails.\".\"You're right,\" said the man. \"You're a little bit of a bitch, right?\".\"No,\" said the woman. \"I'm going to be the one to get started. I'm going to be the one to make you happy. I'm going to be the one to go out and grab a drink and have a few beers and then I'm going to start dancing and doing this. And then I'm going to be the one to stop and look at you and say, 'Oh my god, you're a fucking idiot. You're such a pussy.' I'm going to have a few drinks and then I'm going to have some more drinks and then I'm going to have a few more beers and then I'm going to be the one to leave and go home and have a few more beers. And then I'm going to have a few more drinks and then I'm going to have some more beers and then I'm going to have some more beers and then I'm going to be the one to leave and go home and have a few more beers",
    "Send the stuff in a thick paper bag.The first thing you need to do is to get the papers out of the bag. You can get them in a cardboard box or a plastic bag. Put the papers in a large container.Then you want to put the papers in the paper bag. Put the paper bag on top of the bag. Open the paper bag and move the paper bag around so that the paper is in the bag.The paper bag should be in the bag.Move the paper bag around so that it is on top of the bag.The paper bag should be in the bag.Move the paper bag around so that it is on top of the bag.Move the paper bag around so that it is on top of the bag.Place the paper bag on top of the bag.Put the paper bag on the ground.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on top of the bag.Move the paper bag on",
    "Time brings us many changes.One of the most exciting changes is the creation of a new level of customization in the game. The level of customization can be changed in three ways:.1. You can customize your character.2. You can customize your weapon.3. You can customize your armor.There are many new things that can be done with this new customization. For example, you can change the color of your armor.There are many more things that can be done with this new customization. For example, you can change the color of your armor. You can also assign some of the customization to your character.You can even create unique items for your character.You can even change the color of your armor. The new level of customization allows you to change the armor stats of your character and increase its armor quality (e.g. more damage, armor durability, or magic resistance).You can even change the armor stats of your character and increase its armor quality (e.g. more damage, armor durability, or magic",
    "Take the match and strike it against your shoe. If you are not careful, you will lose in a matter of seconds.If you are the best at it, you can use a second strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a second strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.If you are the best at it, you can use a third strike to make sure you are not in front of your opponent's feet.",
    "A thin stripe runs down the middle. A long, round, circular stripe runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A circle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A circle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down the middle. A triangle runs down",
    "The couch cover and hall drapes were blue. The only thing that made it different was the fact that it was made of a very heavy concrete floor. The couch cover was made of a very heavy concrete, and the hall drapes were made of a very heavy concrete.The couch cover was made of a very heavy concrete, and the hall drapes were made of a very heavy concrete. The floor was made of a very heavy steel. The floor was made of steel.The floor was made of steel. The floor was made of steel. The floor was made of steel. The floor was made of steel.The floor was made of steel. The floor was made of steel. The floor was made of steel. The floor was made of steel.The floor was made of steel. The floor was made of steel. The floor was made of steel. The floor was made of steel.The floor was made of steel. The floor was made of steel. The floor was made of steel.The floor was made of steel. The floor was made of steel. The floor was made of steel. The floor was made of steel.The floor was made of steel. The floor was made of steel.",
    "After the dance, they went straight home. The next time they saw each other, they went to see a movie. They had a little laugh. I remember saying, \"I'm not sure if I'm going to be able to do this, but I'm not going to be able to do this.\" I'm not sure if I'm going to be able to do this. And I'm not sure if I'm going to be able to do this. I don't know if I'm going to be able to do this. I don't know if I'm going to be able to do this. I don't know if I'm going to be able to do this.A: So, what do you think?.J: I think that's really cool. I feel like I'm pretty much in the same boat. But I think it's just the way I see things. I can't really control what I see, because I'm just so focused on how I see things. And I don't really know what I'm going to do with my life. I don't know what I'm going to do with my life. I just keep going.A: Any time you go to a movie, do you want to go back to your old life?.J: I don't really know if I want to do that, but I don't know if I want to do that. I don't really know. I only know",
    "Send the stuff in a thick paper bag. Keep the lid on, cover it with plastic wrap, and put it back in.I'm not sure if I can get this to work without a few things. The only thing I can think of is that it's pretty hard to take apart and clean it. Also, if you're using this as a starter, if you're using it as a first, you'll likely have to use your hand to remove it.I used this to clean the entire top of the table. I've never seen it do this before.I used to do this a lot on my kitchen table. It was the only time I had to do it. I don't know if it's ever done that much, but I'm sure it will.I've never seen a person try to do this.I've never seen people try to do this.I've never seen a person try to do this.I've never seen people try to do this.I've never seen people try to do this.I have a whole collection of homemade table towels.My favorite part of the process is the process of drying the towels. I don't know if there's a better way to do it, but I think that's pretty simple.I've used this on a few different dishes, but this is",
    "Take the winding path to reach the lake.Step 2: Step 2, turn left to the lake. If you're looking to climb to the lake, turn left.Step 3: Turn left to the lake. If you're looking to climb to the lake, turn left.Step 4: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 5: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 6: Turn left to the lake. If you're looking to climb to the lake, turn right.Step 7: Turn left to the lake. If you're looking to climb to the lake, turn left.Step 8: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 9: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 10: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 11: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 12: Turn right to the lake. If you're looking to climb to the lake, turn right.Step 13: Turn left to the lake. If you're looking to climb to the lake, turn left.Step 14:",
    "The thaw came early and freed the stream.\"This is the last time we'll see you,\" he said.\"I had some other plans, and we'll see how you do,\" the woman said.They drove back to the town and took a break. Then the boat started to come down again.They made their way to the town, where they found a few more refugees waiting for them.\"They looked like they'd been shot,\" said a woman who lived in a house in the village. \"They had green eyes and hair that were not red.\".The refugees told the men to leave the village. The women told the men to go home.As the men started to walk away, the woman heard another door open.\"That's where we're going,\" she said. \"We're going to the town. We're going to have to fight.\".As they walked out of the town, the men stopped in front of the house where they had been hiding.\"They were crying,\" said a woman who lives in a house in the village. \"They said, 'We're going back to the village. We're going to have to fight.'\".The refugees told the men to go back to the village.The men told the men to go home.The women",
    "The thaw came early and freed the stream.\"I got up and ran home and I saw my wife and child in the yard and I said 'I'm sorry, this is not the right time to go out and take a nap, I'm going to go out and sleep like this'.\".\"I heard a commotion and I saw the police officer come out and the family were screaming and crying, I heard them go on and on. And I heard them leave.\"I heard the police officer come down the stairs and I saw him in the bedroom, he was looking at my wife and children who were crying and screaming.\"He said 'I don't know what to do, I think you're in trouble, I'm not going to give you a ticket, I've got to get to the police station, I've got to get you a ticket, I've got to get you to the police station and I know what you're going through'.\".He added: \"I know what you're going through, it's a very difficult time.\"I know you're in fear, I know you're in a very difficult situation.\"I'm just trying to help. I'm just trying to help, I'm just trying to help as much as possible, I'm just trying to help, but I'm just trying to",
    "The heart beat strongly and with firm strokes. The heart's core is not as strong as it might be, but it is still strong. The main muscle is the ventricle, the muscle that protects the heart and the heart muscle that protects the lungs and the heart valves. The heart is not as strong as it could be, but it is still strong.The heart beats at a rate of about 60 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats per minute, which means that the heart is beating at about 1.5 beats",
    "The bill was paid every third week.\"It's a great bill,\" said the Republican lawmaker, who was not authorized to speak publicly about the bill.Democrats, meanwhile, blasted the bill as a \"total waste of money\" and said the bill would cost taxpayers $15 billion over 15 years.\"The bill will make it harder for families to afford the care they need, and leave families without care for their children and grandchildren,\" said Democratic Sen. Al Franken (D-Minn.). \"We're not going to stop this bill. We're going to get it done.\".The House has approved $1.5 billion in spending cuts to the federal government, including $700 million for a new border wall along the U.S.-Mexico border.The lawmakers on the House Budget Committee have sent a letter to the president calling for the Senate to pass the bill.The bill's supporters say the federal government will raise the debt limit by $1.5 trillion by this year.The House has also passed a bill to raise the debt limit by $1.2 trillion by 2018, and has passed a bill to cut the government's revenue by $3.7 trillion",
    "After the dance, they went straight home.\"I was too tired to go,\" she said. \"I went to the bathroom and it was like, 'Oh my God.'\".But she found a job as a delivery girl on a construction site. The next morning she was back at work, and she was excited.\"I was like, 'I just want to go home,'\" said Ms. Mone. \"I was like, 'Wow, I could do this.'\".Ms. Mone said she has been working in construction for 10 years. She said she's been working for about 4,000 hours in the last seven years.\"It's amazing,\" she said. \"I'm so happy that I'm doing this.\".That's why the nonprofit has started the website Nelly's Workplace, which has more than 1,200 jobs to open in a dozen cities across the country.\"We're looking for a lot of people who are able to do what they love and do what they love,\" said Ms. Mone. \"It's really a testament to the ability of workers to be able to do what they want.\".Ms. Mone said she's been working at Nelly's for about 20 years. She said she's a little nervous about what she might see next.\"I just want to keep going,\" she said. \"I want to see what I can do",
    "They took the axe and the saw to the forest. They found the body of a young man. He had been shot in the back. The police took him to the hospital. He was taken to the hospital, where he died. We asked them to try and find the body, but they refused. We tried to find a man, but they said that they had to take him to the hospital. We took him to the hospital and tried to find a man. He was taken to the hospital. We asked them to try and find the body, but they refused. We tried to find a man, but they said that they had to take him to the hospital. We took him to the hospital. We tried to find a man. He was taken to the hospital, where he died. We asked them to try and find the body, but they refused. We tried to find a man, but they said that they had to take him to the hospital. We took him to the hospital, where he died. We asked them to try and find the body, but they refused. We tried to find a man, but they said that they had to take him to the hospital. We took him to the hospital, where he died. We asked them to try and find the body, but they refused. We tried to find a man",
    "Every word and phrase he speaks is true. So, how do we know it's true?.Because it's true.You know, the word 'truth' is often used by people who aren't Christians or atheists or agnostics to mean something like, 'I believe that there is a God, but I don't believe in Him.' Well, there are a lot of people who don't believe in God, and then there are a lot of people who believe in God and then there's a lot of people who don't believe in God, and so on and so forth, and so forth.But that's not how it works.You know, there are people who believe in God, but they don't believe in Him.So, I think that what we need to be aware of is that we don't know what God is or what he means, because we don't know what it means. And then we need to be aware of the fact that we don't know what it means to be human. And then we need to be aware of the fact that we don't know what it means to be human.And then we need to be aware of the fact that we don't know where we come from, or where we are going.And then we need to be aware of the fact that the Bible says that to be human is to be born male, and that's",
    "The bill was paid every third week.\"This is a huge step forward for the LGBTQ community in Colorado,\" said Raul Ruppert, executive director of Equality Colorado. \"The bill puts Colorado in the forefront of a national effort to make sure the rights of all of our LGBTQ citizens are protected.\".The bill would allow Colorado to keep transgender people out of the workforce and to allow employers to deny employment to same-sex couples.The bill was sponsored by Rep. John E. McAlister (R-Cooper), who was the top Democrat on the House Judiciary Committee last year.The bill, which was sponsored by Rep. Mike J. Gannon (R-Plano), will be voted on by the House Judiciary Committee in January.Supporters say the bill will give transgender people a way to express their gender identity without fear of discrimination.\"It's a real step forward for our LGBTQ community and the people of Colorado,\" said Steve Spindler, executive director of Equality Colorado.The bill would allow employers to deny employment to same-sex couples.The bill also would allow Colorado to keep transgender people out of school and into retirement.The",
    "The thaw came early and freed the stream.\"I'm still not sure how I got here, but I think I came here by accident,\" he said. \"I was driving my car on a bridge and the dam started to come down. The stream was coming down like a rocket. I got a little bit of water, so I went and started digging. I started digging, but I didn't get any water. I guess I just couldn't get anything up into the water, so I started digging a lot more.\"I was about two miles out of the water, and I heard a sound and saw a big stream coming down. I thought maybe it was a boat. I saw a big stream coming down and I said, 'Well, that's probably the next river. It's probably the next river.' I looked down and saw a large stream coming down from the river. I called a helicopter and I saw a big stream coming down from the river. I said, 'Well, I've got to get the water out of there.' I couldn't get it out of the water. I went down the river. I saw a big stream coming down. I went down the stream. I saw a big stream coming down. I went down the stream. I got the water out of there. I got the water out of there. I got it up over there and",
    "Every word and phrase he speaks is true.\"I'm taking a leave from work. I'm going to take a leave from all this. I'm going to take a leave from this place. I'm going to take a leave from all this. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place. I'm going to take a leave from this place.",
    "Take the match and strike it against your shoe.The match is about to begin with the best of the best. Every single character on this team is going to be on fire. The players have been playing for so long that all of them have been playing on the same team. The match is about to start with one of those big games.The next game will take place before the match starts. It's going to be a high level team battle, where the enemy team can take a series of shots. You will be able to easily do this with a single shot, with each shot coming after the other.The main reason that this game is so long and difficult is because the enemy team is so skilled and ready to take shots. So many enemies are being sent out in the air and are going to be coming after you. The other two teams that are not really going to be able to take shots are the ones that are playing the same game and are all on the same team.You will see different things on the map when you start the game. You will see the enemy team coming after you and the other team will come after you.You will also see different things. Some people say that the enemy team is going to be able to take a series of",
    "Take the match and strike it against your shoe.\"I am very pleased with the results. The players have been terrific. I am very pleased with the results. The players have been terrific. I am very pleased with the results. The players have been terrific. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am very pleased with the results. I am",
    "Act on these orders with great speed.1. The following is a list of all orders received from the following sources:.1. The following is a list of all orders received from the following sources:.2. The following is a list of all orders received from the following sources:.3. The following is a list of all orders received from the following sources:.4. The following is a list of all orders received from the following sources:.5. The following is a list of all orders received from the following sources:.6. The following is a list of all orders received from the following sources:.7. The following is a list of all orders received from the following sources:.8. The following is a list of all orders received from the following sources:.9. The following is a list of all orders received from the following sources:.10. The following is a list of all orders received from the following sources:.11. The following is a list of all orders received from the following sources:.12. The following is a list of all orders received from the following sources:.13. The following is a list of all orders received from the following sources:.14. The following is a list of all orders received from the",
    "Take the winding path to reach the lake.The lake takes two steps. The road takes a short distance. The bridge takes a long distance. The wind blows down the bridge and the water flows back down.The lake is an ideal place to get to meet with a friend or family member.The lake is about a half mile long and spans about three miles. The lake is well-marked with a white fence and a small pond. The lake is very well-lit and the water is clear.The lake is a great place to get water skiing or water climbing. Please keep in mind that the lake is a very hot summer day and the water temperature is very cold.The lake is a great place to get to see nature, enjoy the views and enjoy the views of nature.The lake is a great place to get to see the sunrise and sunset.The lake has a beautiful view of the mountains.The lake is a great place to get to see nature, enjoy the views and enjoy the views of nature.The lake is a great place to get to see nature and enjoy the views and enjoy the views of nature.The lake is an ideal place to get to see the sunrise and sunset.The lake is a great place to get to see nature, enjoy the views and enjoy the views of nature.The lake is a great place to get to see nature, enjoy the views and enjoy",
    "Every word and phrase he speaks is true. He is just like God. He is the word of God. He is the very substance of God. He is the very essence of the universe. He is the very most perfect of all beings. He is the very essence of the Universe. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the very essence of all things. He is the",
    "Take the winding path to reach the lake. The boat will get stuck in the water and will have to break free of the rope.Walk up the side of the lake and you will see a large lake. Turn right and you will see a large lake. Turn left and you will see a large lake. Turn right and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn left and you will see a large lake. Turn",
    "We are sure that one war is enough. We are sure that one nation is going to do what is right for our country. We are sure that one nation is going to do what is right for our children.\".The U.S. military has been in Iraq and Afghanistan since 2003. The U.S. is currently in the process of redeploying troops to Iraq and is planning to conduct more than one mission there.The Pentagon has also been in contact with the Taliban, which has been on the receiving end of U.S. airstrikes in Afghanistan.\"It's very clear that there is a desire to escalate the situation in Afghanistan and we are pleased to see that the United States has been very clear on that,\" said Pentagon spokeswoman Heather Nauert.In 2014, the Pentagon released a statement saying that the U.S. was \"deeply concerned\" about the situation in Afghanistan.\"The Taliban and their affiliates in Afghanistan and Pakistan have committed war crimes against the United States and our allies in the region and for that they have been deeply disappointed,\" the statement said.The U.S. military has been conducting strikes in Afghanistan since 2001, when it began a four-month mission to defend the country from the Taliban.During the campaign, the U.S. conducted two air strikes in Afghanistan with the aim",
    "Act on these orders with great speed.We will continue to make the necessary preparations in the interests of safety and to carry out the tasks under our command of military forces, which are to be carried out in accordance with our instructions and will be carried out in accordance with the provisions of the Act and our laws.We will continue to carry out the duties of the Army of the Philippines and the Philippine Army, which are to be carried out in accordance with our instructions and will be carried out in accordance with our laws, and will be provided for by the Army of the Philippines, which is to be carried out in accordance with our orders and will be provided for by the Army of the Philippines, which is to be carried out in accordance with our laws, in accordance with law and regulations, and in accordance with the provisions of the Act and the laws, and the provisions of the law, and we will uphold the rights of all Filipinos and the Philippine people to life, liberty, property and the pursuit of happiness.We will continue to carry out the duties of the Army of the Philippines and the Philippine Army, which are to be carried out in accordance with our orders and will be provided for by the Army of the Philippines, which is to be carried out in accordance with our orders, in accordance with our laws, and the provisions of the Act"
]
import random
random.shuffle(texts)

In [13]:
# texts = {
#   'LJ016-0274.wav': "and the windows of the opposite houses, which commanded a good view, as usual fetched high prices .",
#   'LJ050-0022.wav': "A formal and thorough description of the responsibilities of the advance agent, is now in preparation by the Service .",
#   'LJ018-0159.wav': "This was all the police wanted to know .",
#   'LJ029-0004.wav': "The narrative of these events is based largely on the recollections of the participants ,",
#   'LJ046-0191.wav': "it had established periodic regular review of the status of four hundred individuals ;",
#   'LJ016-0054.wav': "but he did not like the risk of entering a room by the fireplace, and the chances of detection it offered .",
#  'LJ038-0035.wav': "Oswald rose from his seat, bringing up both hands.",
#  'LJ013-0055.wav': "The jury did not believe him, and the verdict was for the defendants .",
#  'LJ021-0139.wav': "There should be at least a full and fair trial given to these means of ending industrial warfare;",
#  'LJ002-0260.wav': "Yet the public opinion of the whole body seems to have checked dissipation .",
#  'LJ023-0033.wav': "we will not allow ourselves to run around in new circles of futile discussion and debate, always postponing the day of decision .",
#  'LJ018-0206.wav': "He was a tall, slender man, with a long face and iron-gray hair .",
#  'LJ019-0055.wav': "High authorities were in favor of continuous separation.",
#  'LJ010-0027.wav': "Nor did the methods by which they were perpetrated greatly vary from those in times past.",
#  'LJ021-0012.wav': "with respect to industry and business, but nearly all are agreed that private enterprise in times such as these",
#  'LJ021-0025.wav': "And in many directions, the intervention of that organized control which we call government .",
# #  'LJ021-0140.wav': "and in such an effort we should be able to secure for employers and employees and consumers.",
#  'LJ048-0200.wav': "paying particular attention to the crowd for any unusual activity .",
#  'LJ018-0349.wav': "His disclaimer, distinct and detailed on every point, was intended simply for effect .",
#  'LJ046-0055.wav': "It is now possible for presidents to travel the length and breadth of a land far larger than the United States .",
#  'LJ008-0215.wav': "One by one the huge uprights of black timber were fitted together",
#  'LJ021-0040.wav': "The second step we have taken in the restoration of normal business enterprise",
#  'LJ006-0021.wav': "Later on he had devoted himself to the personal investigation of the prisons of the United States.",
#  'LJ028-0421.wav': "It was the beginning of the great collections of Babylonian antiquities in the museums of the Western world.",
#  # 'LJ002-0225.wav': "The rentals of rooms and fees went to the warden, whose income was £2,372.",
    
#  'LJ034-0035.wav': "The position of this palmprint on the carton was parallel with the long axis of the box, and at right angles with the short axis ;",
#  'LJ015-0025.wav': "The bank enjoyed an excellent reputation, it had a good connection, and was supposed to be perfectly sound .",
#  'LJ003-0282.wav': "Many years were to elapse before these objections should be fairly met and universally overcome.",
#  'LJ016-0367.wav': "Under the new system the whole of the arrangements from first to last fell upon the officers.",
#  'LJ012-0049.wav': "led him to think seriously of trying his fortunes in another land .",
#  'LJ037-0248.wav': "The eyewitnesses vary in their identification of the jacket .",
#  'LJ019-0371.wav': "Yet the law was seldom if ever enforced.",
#  'LJ037-0007.wav': "Three others subsequently identified Oswald from a photograph .",
#  'LJ030-0063.wav': "He had repeated this wish only a few days before, during his visit to Tampa, Florida .",
#  'LJ010-0297.wav': "but there were other notorious cases of forgery .",
#  'LJ050-0029.wav': "that is reflected in definite and comprehensive operating procedures .",
#  'LJ004-0239.wav': "He had been committed for an offense for which he was acquitted .",
#  'LJ035-0155.wav': "The only exit from the office in the direction Oswald was moving was through the door to the front stairway .",
#  'LJ027-0006.wav': "In all these lines the facts are drawn together by a strong thread of unity .",
#  'LJ016-0277.wav': "This is proved by contemporary accounts, especially one graphic and realistic article which appeared in the 'Times,'",
#  'LJ002-0171.wav': "The boy declared he saw no one, and accordingly passed through without paying the toll of a penny .",
#  'LJ033-0072.wav': "I then stepped off of it and the officer picked it up in the middle and it bent so ."
# }

# texts = list(texts.values())

In [14]:
# texts = [
#     "In all these lines the facts are drawn together by a strong thread of unity.",
#     "After the construction and action of the machine had been explained, the doctor asked the governor what kind of men he had commanded at Goree."
# ]

In [15]:
texts = [
    "So , yesterday I went to the zoo , and , you know , it was such an incredible experience ; I mean , like , absolutely breathtaking !" ,
    "I wasn't quite sure what to expect , but , you know , it turned out to be really , really fun ; like , surprisingly enjoyable ." ,
    "Oh , I saw so many amazing animals ; lions , tigers , elephants , you name it ; it was like a wildlife paradise !" ,
    "At first , I thought it might be crowded , but , fortunately , it wasn't too bad ; I guess I got lucky ." ,
    "You know , I'm not usually a fan of large crowds , but somehow , I managed to enjoy myself ; it was quite the adventure ." ,
    "So , when I got there , the first thing I did was head straight to the monkey exhibit ; those little guys are just so fascinating , you know ?" ,
    "Ah , I'm absolutely fascinated by monkeys ; they're so lively and entertaining to watch ; I could spend hours just observing them ." ,
    "I spent quite a bit of time there , just admiring their playful antics and interactions with each other ." ,
    "But eventually , I decided to tear myself away and , like , explore the rest of the zoo ; there was just so much more to see !" ,
    "Oh , I stumbled upon the giraffe enclosure next ; those majestic creatures are truly a sight to behold , don't you think ?" ,
    "I took a ton of pictures of them , but , none of them really did justice to their beauty ; they're just so graceful and magnificent ." ,
    "You know , I'm not exactly the best photographer , but I tried my best to capture their elegance and grandeur ." ,
    "So , after spending some time with the giraffes , I decided to take a little break and , like , grab a snack ." ,
    "Ah , I'm a bit of a sucker for zoo food ; it may not be the healthiest , but , it's definitely delicious ; I just couldn't resist ." ,
    "I indulged in some popcorn and , a soda , and just , you know , sat down for a while , soaking in the vibrant atmosphere around me ." ,
    "But soon enough , I felt re-energized and ready to , like , embark on the next leg of my zoo adventure ; there was still so much to explore !" ,
    "Oh , I stumbled upon the reptile house next ; now that , that was an experience , let me tell you !" ,
    "I have to admit , I'm not exactly fond of snakes , but , I braved my fears and ventured in anyway ; it was quite the adrenaline rush ." ,
    "You know , I'm not entirely sure why I did that to myself , but , it definitely made for an interesting experience , to say the least ." ,
    "So , I cautiously made my way through the exhibit , trying not to , like , jump at every sudden movement or slithering motion ; it was quite the challenge ." ,
    "Ah , I'm proud of myself for confronting my fears head-on , but , you know , I don't think I'll be rushing back to the reptile house anytime soon ; once was probably enough for me ." ,
    "After that rather nerve-wracking experience , I felt the need for something a little more uplifting , so I , like , headed straight for the penguin exhibit ." ,
    "I absolutely adore penguins ; they're just so adorable and , comical , especially when they waddle around on land ; it's simply delightful to watch ." ,
    "I spent a good amount of time there , just enjoying their playful antics and quirky personalities ; they never fail to bring a smile to my face ." ,
    "But eventually , I tore myself away and , like , continued on with my zoo adventure ; there were still so many more exhibits to explore !" ,
    "Oh , I stumbled upon the big cat area next ; now that , that was an exhilarating experience , let me tell you !" ,
    "I have to admit , I was both excited and a little nervous to see the lions and tigers up close ; they're such majestic and powerful creatures ." ,
    "I kept a safe distance , of course , but , just being in their presence filled me with a sense of awe and wonder ." ,
    "But , as the day started to draw to a close , I realized that my time at the zoo was coming to an end ; it was bittersweet , to say the least ." ,
    "Oh , I hadn't realized how quickly the hours had flown by ; I guess I was just having too much fun !" ,
    "So , I reluctantly made my way towards the exit , but not before promising myself that I'd , like , come back again soon ; there was still so much more to see and explore !" ,
    "I felt a profound sense of gratitude for the incredible experience I'd had at the zoo ; it was truly unforgettable , and I couldn't wait to , like , do it all over again !" ,
]

In [16]:
from collections import defaultdict
import json


class LenRTF:
    def __init__(self):
        self.storage = defaultdict(list)
    
    def add(self, txt_len, rtf):
        self.storage[txt_len].append(rtf)
        
    @property
    def data(self):
        return dict(self.storage)
    
    def to_json(self, filename):
        json.dump(self.data, open(filename, "w"))

In [18]:
rtfs, rtfs_w = [], []
# 1, 2, 3, 4, 
for n_timesteps in [50]:

    with torch.no_grad():
        len_rtf = LenRTF()

        for i, text in enumerate(tqdm(texts)):
            print(f"=> {n_timesteps}")
            x = torch.LongTensor(intersperse(text_to_sequence(text, ['english_cleaners2']), 0)).cuda()[None]
            print("Here")
#             import pdb; pdb.set_trace()
            print("Here2", sequence_to_text(x[0].tolist()))
            x_lengths = torch.LongTensor([x.shape[-1]]).cuda()

            start_t = dt.datetime.now()
            y_enc, y_dec, attn = generator.forward(x, x_lengths, n_timesteps=n_timesteps, temperature=1.3,
                                                   stoc=False, spk=None if N_SPKS==1 else torch.LongTensor([15]).cuda(),
                                                               length_scale=0.85)
            t = (dt.datetime.now() - start_t).total_seconds()
            orig_t_am = t
            rtf = t * 22050 / (y_dec.shape[-1] * 256)
            rtfs.append(rtf)

            audio = hifigan.forward(y_dec).clamp(-1, 1)
            audio = denoiser(audio.squeeze(0), strength=0.00025).cpu().squeeze()
            audio = audio.squeeze().cpu()
            if True:
                ipd.display(ipd.Audio(audio, rate=22050))
            print('.', end="")
            t = (dt.datetime.now() - start_t).total_seconds()
            rtf_w = t * 22050 / (audio.shape[-1])

            len_rtf.add(len(text), orig_t_am)
            rtfs_w.append(rtf_w)
            save_to_folder(f"{i}", audio, f'cormac_fm_grad/')
            
            
    len_rtf.to_json(f'GRAD_{n_timesteps}.json')
    print(f"Mean RTF: {n_timesteps} {np.mean(rtfs):.6f} ± {np.std(rtfs):.6f}")
    print(f"Mean RTF W: {n_timesteps} {np.mean(rtfs_w):.6f} ± {np.std(rtfs_w):.6f}")

    
# plt.figure(figsize=(15, 4))
# plt.subplot(1, 3, 1)
# plt.title('Encoder outputs')
# plt.imshow(y_enc.cpu().squeeze(), aspect='auto', origin='lower')
# plt.colorbar()
# plt.subplot(1, 3, 2)
# plt.title('Decoder outputs')
# plt.imshow(y_dec.cpu().squeeze(), aspect='auto', origin='lower')
# plt.colorbar()
# plt.subplot(1, 3, 3)
# plt.title('Alignment')
# plt.imshow(attn.cpu().squeeze(), aspect='auto', origin='lower');

  0%|          | 0/32 [00:00<?, ?it/s]

=> 50
Here
Here2 _s_ˈ_o_ʊ_ _,_ _j_ˈ_ɛ_s_t_ɚ_d_ˌ_e_ɪ_ _ˈ_a_ɪ_ _w_ɛ_n_t_ _t_ə_ _ð_ə_ _z_ˈ_u_ː_ _,_ _æ_n_d_ _,_ _j_u_ː_ _n_ˈ_o_ʊ_ _,_ _ɪ_t_ _w_ʌ_z_ _s_ˈ_ʌ_t_ʃ_ _ɐ_n_ _ɪ_ŋ_k_ɹ_ˈ_ɛ_d_ɪ_b_ə_l_ _ɛ_k_s_p_ˈ_i_ə_ɹ_ɪ_ə_n_s_ _;_ _ˈ_a_ɪ_ _m_ˈ_i_ː_n_ _,_ _l_ˈ_a_ɪ_k_ _,_ _ˌ_æ_b_s_ə_l_ˈ_u_ː_t_l_i_ _b_ɹ_ˈ_ɛ_θ_t_e_ɪ_k_ɪ_ŋ_ _!_


.=> 50
Here
Here2 _ˈ_a_ɪ_ _w_ˈ_ʌ_z_n_̩_t_ _k_w_ˈ_a_ɪ_t_ _ʃ_ˈ_ʊ_ɹ_ _w_ʌ_t_ _t_ʊ_ _ɛ_k_s_p_ˈ_ɛ_k_t_ _,_ _b_ˌ_ʌ_t_ _,_ _j_u_ː_ _n_ˈ_o_ʊ_ _,_ _ɪ_t_ _t_ˈ_ɜ_ː_n_d_ _ˈ_a_ʊ_t_ _t_ə_b_i_ _ɹ_ˈ_i_ə_l_i_ _,_ _ɹ_ˈ_i_ə_l_i_ _f_ˈ_ʌ_n_ _;_ _l_ˈ_a_ɪ_k_ _,_ _s_ɚ_p_ɹ_ˈ_a_ɪ_z_ɪ_ŋ_l_i_ _ɛ_n_d_ʒ_ˈ_ɔ_ɪ_ə_b_ə_l_ _._


.=> 50
Here
Here2 _ˈ_o_ʊ_ _,_ _ˈ_a_ɪ_ _s_ˈ_ɔ_ː_ _s_ˌ_o_ʊ_ _m_ˈ_ɛ_n_i_ _ɐ_m_ˈ_e_ɪ_z_ɪ_ŋ_ _ˈ_æ_n_ɪ_m_ə_l_z_ _;_ _l_ˈ_a_ɪ_ə_n_z_ _,_ _t_ˈ_a_ɪ_ɡ_ɚ_z_ _,_ _ˈ_ɛ_l_ɪ_f_ə_n_t_s_ _,_ _j_u_ː_ _n_ˈ_e_ɪ_m_ _ɪ_t_ _;_ _ɪ_t_ _w_ʌ_z_ _l_ˈ_a_ɪ_k_ _ɐ_ _w_ˈ_a_ɪ_l_d_l_a_ɪ_f_ _p_ˈ_æ_ɹ_ə_d_ˌ_a_ɪ_s_ _!_


.=> 50
Here
Here2 _æ_t_ _f_ˈ_ɜ_ː_s_t_ _,_ _ˈ_a_ɪ_ _θ_ˈ_ɔ_ː_t_ _ɪ_t_ _m_ˌ_a_ɪ_t_ _b_i_ː_ _k_ɹ_ˈ_a_ʊ_d_ᵻ_d_ _,_ _b_ˌ_ʌ_t_ _,_ _f_ˈ_ɔ_ː_ɹ_t_ʃ_ə_n_ə_t_l_i_ _,_ _ɪ_t_ _w_ˈ_ʌ_z_n_̩_t_ _t_ˈ_u_ː_ _b_ˈ_æ_d_ _;_ _ˈ_a_ɪ_ _ɡ_ˈ_ɛ_s_ _ˈ_a_ɪ_ _ɡ_ɑ_ː_t_ _l_ˈ_ʌ_k_i_ _._


...

.=> 50
Here
Here2 _ˈ_ɑ_ː_ _,_ _a_ɪ_m_ _ˌ_æ_b_s_ə_l_ˈ_u_ː_t_l_i_ _f_ˈ_æ_s_ᵻ_n_ˌ_e_ɪ_ɾ_ᵻ_d_ _b_a_ɪ_ _m_ˈ_ʌ_ŋ_k_i_z_ _;_ _ð_e_ɪ_ɚ_ _s_ˌ_o_ʊ_ _l_ˈ_a_ɪ_v_l_i_ _æ_n_d_ _ˌ_ɛ_n_t_ɚ_t_ˈ_e_ɪ_n_ɪ_ŋ_ _t_ə_ _w_ˈ_ɑ_ː_t_ʃ_ _;_ _ˈ_a_ɪ_ _k_ʊ_d_ _s_p_ˈ_ɛ_n_d_ _ˈ_a_ʊ_ɚ_z_ _d_ʒ_ˈ_ʌ_s_t_ _ə_b_z_ˈ_ɜ_ː_v_ɪ_ŋ_ _ð_ˌ_ɛ_m_ _._


.=> 50
Here
Here2 _ˈ_a_ɪ_ _s_p_ˈ_ɛ_n_t_ _k_w_ˈ_a_ɪ_t_ _ɐ_ _b_ˈ_ɪ_t_ _ʌ_v_ _t_ˈ_a_ɪ_m_ _ð_ˈ_ɛ_ɹ_ _,_ _d_ʒ_ˈ_ʌ_s_t_ _ɐ_d_m_ˈ_a_ɪ_ɚ_ɹ_ɪ_ŋ_ _ð_ɛ_ɹ_ _p_l_ˈ_e_ɪ_f_ə_l_ _ˈ_æ_n_t_ɪ_k_s_ _æ_n_d_ _ˌ_ɪ_n_t_ɚ_ɹ_ˈ_æ_k_ʃ_ə_n_z_ _w_ɪ_ð_ _ˈ_i_ː_t_ʃ_ _ˈ_ʌ_ð_ɚ_ _._


.=> 50
Here
Here2 _b_ˌ_ʌ_t_ _ᵻ_v_ˈ_ɛ_n_t_ʃ_u_ː_ə_l_i_ _,_ _ˈ_a_ɪ_ _d_ᵻ_s_ˈ_a_ɪ_d_ᵻ_d_ _t_ə_ _t_ˈ_ɛ_ɹ_ _m_a_ɪ_s_ˈ_ɛ_l_f_ _ɐ_w_ˈ_e_ɪ_ _æ_n_d_ _,_ _l_ˈ_a_ɪ_k_ _,_ _ɛ_k_s_p_l_ˈ_o_ː_ɹ_ _ð_ə_ _ɹ_ˈ_ɛ_s_t_ _ʌ_v_ð_ə_ _z_ˈ_u_ː_ _;_ _ð_ɛ_ɹ_w_ˌ_ʌ_z_ _d_ʒ_ˈ_ʌ_s_t_ _s_ˈ_o_ʊ_ _m_ˌ_ʌ_t_ʃ_ _m_ˈ_o_ː_ɹ_ _t_ə_ _s_ˈ_i_ː_ _!_


.=> 50
Here
Here2 _ˈ_o_ʊ_ _,_ _ˈ_a_ɪ_ _s_t_ˈ_ʌ_m_b_ə_l_d_ _ə_p_ˌ_ɑ_ː_n_ _ð_ə_ _d_ʒ_ᵻ_ɹ_ˈ_æ_f_ _ɛ_ŋ_k_l_ˈ_o_ʊ_ʒ_ɚ_ _n_ˈ_ɛ_k_s_t_ _;_ _ð_o_ʊ_z_ _m_ɐ_d_ʒ_ˈ_ɛ_s_t_ɪ_k_ _k_ɹ_ˈ_i_ː_t_ʃ_ɚ_z_ _ɑ_ː_ɹ_ _t_ɹ_ˈ_u_ː_l_i_ _ɐ_ _s_ˈ_a_ɪ_t_ _t_ə_ _b_ᵻ_h_ˈ_o_ʊ_l_d_ _,_ _d_ˈ_o_ʊ_n_t_ _j_u_ː_ _θ_ˈ_ɪ_ŋ_k_ _?_


KeyboardInterrupt: 

In [ ]:
len_rtf.to_json(f'Time_{n_timesteps}.json')


In [14]:
n_timesteps

10